## Predicción de indicadores sociales para países del MERCOSUR - Grupo I



[Link a la descripción del proyecto](https://github.com/jfrau/DiploDatos2020/blob/master/README.md)

## Introducción

 En el presente informe se analizan de forma exhaustiva los datos de variables relacionadas a diferentes indicadores sociales relevados por el Banco Mundial. La base de datos utilizada es de  libre acceso y puede descargarse del siguiente [link](https://drive.google.com/drive/folders/1NVxgmk_pED1FtnqWl_M4-2vRPiyd6Pri?usp=sharing).

 Los indicadores seleccionados para el estudio abarcan las dimensiones de Educación, Género, Medio Ambiente, Pobreza, Protección Social y Salud para el período 1960-2019 de los paises pertenecientes al Mercosur.

 Los países en análisis son: Argentina, Brasil, Paraguay, Uruguay, Venezuela, Bolivia, Chile, Colombia, Ecuador, Guyana, Perú y Suriname.

El objetivo final de la investigación es predecir el valor del año 2019 de los indicadores para cada país. Es por ello que en esta primera aproximación, se realiza un análisis profundo de la base de datos para corroborar si existe suficiente información para lograr el objetivo.

 En el siguiente repositorio, en primera instancia se realizan las importaciones de la librerias y bases de datos correspondientes. Luego, se construye un dataset único y se realiza un breve análisis de la información en forma general. Por último, se analiza de forma exhaustiva cada una de las dimensiones sociales a considerar.


 El proceso de análisis que se sigue para cada dimensión social es el mismo. Primero, se realiza un análisis por feature para toda la región, luego de seleccionar aquellos features con mayor información, se analizan de forma abarcativa los features seleccionados por país con el propósito de determinar si los features muestran comportamientos similares entre los paises y si existe suficiente información para predecir y utilizarlo como prueba al año 2019. Para eso se hace uso de técnicas de análisis y visualziacion de datos: heatmap, histogramas, tablas, boxplot, gráficos de barras y de líneas.

## Importacion de Librerías

In [ ]:
import pandas as pd
!pip install sidetable
import sidetable


#Visualización
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np

In [ ]:
# Para que pandas no trunque aquellas celdas con contenido muy largo (como textos)
pd.set_option('display.max_colwidth', -1)
# Para que pandas no trunque la vista de todas las columnas
pd.set_option('display.max_columns', None)


In [ ]:
import sys
in_colab = 'google.colab' in sys.modules

if in_colab:
  BASE_DIR = "https://raw.githubusercontent.com/sanmope/MentoriaMercosur/master/"
else:
  BASE_DIR = ".."


Cargamos los datasets de cada uno de los indicadores a predecir. 

## Lectura de archivos


In [ ]:
education =  pd.read_csv(BASE_DIR + '/educacion_mercosur_cleaned.csv')
gender = pd.read_csv(BASE_DIR + 'gender_mercosur_cleaned.csv')
environment = pd.read_csv(BASE_DIR + 'environment_mercosur_cleaned.csv')
health = pd.read_csv(BASE_DIR + 'health_mercosur_cleaned.csv')
poverty = pd.read_csv(BASE_DIR + 'poverty_mercosur_cleaned.csv')
social_protection = pd.read_csv(BASE_DIR + 'socialprotection_mercosur_cleaned.csv')

Renombramos la primera columna como ***Features*** en todos los datasets.

In [ ]:
#Corrección del nombre de la primera columna
education = education.rename(columns={"Unnamed: 0": "features"})
gender = gender.rename(columns={"Unnamed: 0": "features"})
environment = environment.rename(columns={"Unnamed: 0": "features"})
health = health.rename(columns={"Unnamed: 0": "features"})
poverty = poverty.rename(columns={"Unnamed: 0": "features"})
social_protection = social_protection.rename(columns={"Unnamed: 0": "features"})

Por último veamos las dimensiones de cada uno de los datasets considerados:

In [ ]:
print('Dimensiones de los datasets: \n',
      'Educación:',education.shape, '\n', 
      'Género:',gender.shape,'\n',
     'Medio Ambiente:', environment.shape, '\n',
     'Salud:', health.shape, '\n',
      'Pobreza:',poverty.shape, '\n',
      'Protección Social:',social_protection.shape)

Es posible acceder a la metadata de las bases consideradas a través del siguiente [link](https://drive.google.com/drive/folders/11QPQHD_hxeMZZ-VH9ppM9KXL46HRxSjy?usp=sharing).

La metadata es una explicación breve sobre el significado de cada variable y la fuente de datos, es un "Diccionario de datos". Estos datos deben ser tomados de consulta y para dar sentido a los valores y/o información hallada.

## Breve Descripción del Dataset Unificado

Con el propósito de tener una base de datos consolidada para utilizar durante el proceso de investigación, se construye un dataset único.

In [ ]:
dataset =pd.concat([education, gender, environment, health, poverty, social_protection])
print ( "Considerando toda la información de forma conjunta, la base de datos con la que se realiza la mentoría \n tiene el siguiente tamaño: "+ str( dataset.shape))

In [ ]:
print("El dataset esta constituido por las siguientes columnas: ")

dataset.columns

In [ ]:
dataset.head(3)

### Features

In [ ]:
print( "En la base de datos consolidada existen: " + str(len(dataset['features'].unique())) + " features en total")

In [ ]:
featureByIndicator = dataset[["features", "Indicator"]].drop_duplicates().groupby('Indicator')['features'].count()

print( "La cantidad de features que existen por indicador es:")
featureByIndicator

In [ ]:
featureByIndicator = dataset[["features", "Indicator"]].drop_duplicates().groupby('Indicator')['features'].count().reset_index()
_ind_max = featureByIndicator[featureByIndicator["features"] == featureByIndicator["features"].max()]
print("El indicador con mayor cantidad de features es: ")
_ind_max


In [ ]:
featureByIndicator = dataset[["features", "Indicator"]].drop_duplicates().groupby('Indicator')['features'].count().reset_index()
_ind_min = featureByIndicator[featureByIndicator["features"] == featureByIndicator["features"].min()]
print("El indicador con menor cantidad de features es: ")
_ind_min

### Países

In [ ]:
print( "En la base de datos consolidada existen: " + str(len(dataset['Country Name'].unique())) + " países en total")

In [ ]:
print( "Los países consdierados en el ánalisis son los siguientes:  ")
print(dataset["Country Name"].unique())

In [ ]:
featureByCountry = dataset.groupby('Country Name')['features'].count()
print("La cantidad de features por país para todo el dataset es: ")
print(featureByCountry)

Podemos observar que existen la misma cantidad de features por cada pais.

Una vez realizada una breve descripción de los datos que componen el dataset consolidado, se procede a realizar un análisis mas detallado por indicador, en el siguiente orden:


*   Género
*   Salud
*   Educación
*   Protección Social
*   Pobreza
*   Medio Ambiente



## GÉNERO

1. ¿Cuántos **tipos de Features diferentes** existen por indicador considerado? ¿Todos los países mantienen esa misma cantidad para el mismo indicador?

In [ ]:
print("El indicador de género tiene " + str(len(gender["features"].unique())) + " features.")
print("El análisis se lleva a cabo consdierando " + str(len(gender["Country Name"].unique())) + " países.")
print(gender["Country Name"].unique())

In [ ]:
countries = gender['Country Name'].unique()

for countries in countries:
    
    print("La cantidad de features que posee " + str(countries) + ": " + str(gender[gender['Country Name']==countries]['features'].count()) )

### Análisis por Feature para toda la Región

---



2.  ¿Cuál es la **proporción de valores nulos** para cada Feature dentro de cada Indicador con respecto al intervalo temporal considerado? ¿Estos valores nulos se encuentran dispersos a lo largo de todo el intervalo temporal o existen agrupamientos (más chicos y/o más grandes) de los mismos? ¿Qué decisión se podría tomar con respecto a los mismos en un futuro?
3. ¿Cómo es la distribución de valores nulos por año (por columnas del dataset)?. Teniendo en cuenta que el objetivo final de la mentoría es realizar una predicción de cada Feature para el año 2019, ¿se disponen de datos suficientes para comparar las predicciones que se obtendrán con el valor real en ese año?

In [ ]:
print("Los features de género que existen son los siguientes: \n", str(gender['features'].unique()))

Para llevar a cabo el análisis de los datos nulos, se construye una matriz por feature y por año en la que se muestra el porcentaje de registros existentes para cada combinación considerando toda la región.

In [ ]:
GenderValuesByICode = gender.groupby("Indicator Code").count().drop(["Country Code", "Country Name", "Indicator", "features"], axis=1)
GenderValuesByICode["Valor"] = GenderValuesByICode.sum(axis=1)
GenderValuesByICode = GenderValuesByICode.sort_values(by=['Valor'], ascending=False)
GenderValuesByICode = ((GenderValuesByICode/12)*100).drop(columns = ["Valor"])
GenderValuesByICode = GenderValuesByICode.rename(columns = lambda x : str(x)[:4])
round(GenderValuesByICode,2)


Se construye un mapa de calor para visualizar aquellas features con mayor porcentaje de registros, mientras mas oscuro el color menos proporción de registros hay para dicha combinación.

In [ ]:
plt.figure(figsize=(60,30))
sns.heatmap(GenderValuesByICode, cbar=True, linewidths=0.1,  vmin=0, vmax=100)
sns.set(font_scale=4)

Como puede observarse graficamente, durante el periodo de 1970 a 1990 se concentra la mayor cantidad de nulos.

In [ ]:
GenderByYear = pd.melt(gender.drop(["Country Code", "Country Name", "Indicator",
                            "features"], axis=1), id_vars = ["Indicator Code"])
GenderByYear["variable"] = pd.to_numeric(GenderByYear["variable"] )
GenderByYear = GenderByYear.rename(columns = {"variable": "Year"})
TotalData_1990 = (GenderByYear["Year"]<=1990).sum()
WithData_1990 = ((GenderByYear["Year"]<=1990) & (GenderByYear["value"]>0)).sum()

print( "El % de registros sin datos para el período de 1970 a 1990 es de: \n" 
      +  str(round((1 - (WithData_1990/TotalData_1990))*100,2)) + "%") 

TotalData_2019 = (GenderByYear["Year"]>1990).sum()
WithData_2019 = ((GenderByYear["Year"]>1990) & (GenderByYear["value"]>0)).sum()
Check = len (GenderByYear) == (TotalData_2019 + TotalData_1990 )


print( "El % de registros sin datos para el período de 1991 a 2019 es de: \n" 
      +  str(round((1 - (WithData_2019/TotalData_2019))*100,2)) + "%") 


Se seleccionan en un principio los 30 features con mas datos en toda la región:

In [ ]:
GenderValuesByICode = GenderValuesByICode.reset_index()
GenderValuesByICode["features"] = GenderValuesByICode["Indicator Code"]
GenderValuesByICode["features"] = GenderValuesByICode["features"].map(gender.set_index("Indicator Code")["features"].to_dict())
FeaturesGenderSelection = GenderValuesByICode.iloc[0:30][["Indicator Code", "features"]].set_index("Indicator Code")
FeaturesGenderSelection

En una primera instancia, dado que la dimensión en estudio es el género, se considera necesario seleccionar como variables de estudio aquellas que representen tanto la situación de los hombres como de las mujeres. Es decir, si se considera el indicador "Relación entre empleo y población, mayores de 15 años, mujeres (%)", entonces se seleccionará para el análisis la misma variable para hombres. Es por eso que se amplia la cantidad de features de punto de partida relevantes para abarcar ambos sexos.

In [ ]:
IDCodeSelection = GenderValuesByICode.iloc[0:30].reset_index()
IDCodeSelection = IDCodeSelection["Indicator Code"].to_list()
AddIDCode = ["SE.PRM.ENRL.MA.ZS", "SE.SEC.ENRL.GC.MA.ZS", "SL.TLF.TOTL.MA.ZS", "SL.EMP.WORK.FE.ZS", "SL.EMP.VULN.MA.ZS", "SE.SEC.ENRL.VO.MA.ZS", "SE.SEC.ENRL.MA.ZS"]
GenderIDCode =  IDCodeSelection + AddIDCode

GenderValues = GenderValuesByICode[GenderValuesByICode["Indicator Code"].isin(GenderIDCode)]

GenderValues = GenderValues.reset_index()
GenderValues["features"] = GenderValues["Indicator Code"]
GenderValues["features"] = GenderValues["features"].map(gender.set_index("Indicator Code")["features"].to_dict())
FeaturesGender = GenderValues[["Indicator Code", "features"]].set_index("Indicator Code")

print("Se agregaron "+ str(len(FeaturesGender)- len(FeaturesGenderSelection)) + " features al listado original. Quedando en total " + str(len(FeaturesGender)) + " features." )
FeaturesGender

### Análisis por País para cada Feature

4. Volviendo a los resultados del punto 2, elegir como mínimo 15 Features por Indicador con al menos el 50% de valores no nulos (y en lo posible los mismos para todos los países) y mostrar gráficamente, con gráficos de distribución o de cajas por ejemplo, la distribución de los mismos. ¿Qué se puede observar de estas distribuciones? ¿Son normales? ¿Qué sucede con los outliers? ¿Observan alguna similitud entre los features elegidos?. Para estos mismos 15 Features realizar el gráfico de línea de su serie temporal y analizarlo.

Se construye una matriz para determinar cuál es el porcentaje de registros con datos considerando las 32 features seleccionadas por país y año. Es decir, por ejemplo, para Chile en el año 1970 el 65.6% de los features tiene valores.

In [ ]:
RelevantFeaturesGender = gender[gender["Indicator Code"].isin(GenderIDCode)]
GenderDataByCountry = RelevantFeaturesGender.groupby("Country Name").count().drop(["features", "Indicator", "Country Code", "Indicator Code"], axis=1)
GenderDataByCountry["Valor"] = GenderDataByCountry.sum(axis=1)
GenderDataByCountry = GenderDataByCountry.sort_values(by=['Valor'], ascending=False)
GenderDataByCountry = ((GenderDataByCountry/len(RelevantFeaturesGender["Indicator Code"].unique()))*100).drop(columns = ["Valor"])
GenderDataByCountry = GenderDataByCountry.rename(columns = lambda x : str(x)[:4])
round(GenderDataByCountry,2)

In [ ]:
plt.figure(figsize=(40, 20))
sns.heatmap(GenderDataByCountry, cbar=True, linewidths=.3, vmin=0, vmax=100)
sns.set(font_scale=4)

Se realiza un heatmap considerando como centro el 50% de esta forma se puede visualizar cuáles son las combinaciones que tienen mas del 50% de registros con datos.

In [ ]:
plt.figure(figsize=(40, 20))
sns.heatmap(GenderDataByCountry, cbar=True, linewidths=.3, center=50, cmap="PRGn")
sns.set(font_scale=4)

In [ ]:
_porc_paises_mas_50 = round((GenderDataByCountry[GenderDataByCountry>50].count()/len(gender["Country Name"].unique()))*100, 0).astype(int)

print("El % de países con más del 50% de los features con datos para cada año es: \n" + str(_porc_paises_mas_50))

A partir de 1991, todos los paises en análisis tienen información para más del 50% de los features relevantes. Esto se evidencia en el heatmap, donde aquellas combinaciones con más del 50% de registros se muestran en diferentes tonalidades de verdes. Aquellas combinaciones que no alcanzan el 50% de los registros tienen diferentes tonalidades de violeta.

Se modifica la estructura del dataset para poder analizar el porcentaje de registros con datos por país, por feature y por año, para decidir qué features considerar para cada país.

In [ ]:
GenderFeaturesByID = RelevantFeaturesGender.drop(["Country Code", "Indicator", "features"], axis=1)
GenderFeaturesByID = GenderFeaturesByID.groupby(["Country Name", "Indicator Code"]).count()
GenderFeaturesByID["Porcentaje de Registros"] = (GenderFeaturesByID.sum(axis=1)/50)*100
GenderFeaturesByID = GenderFeaturesByID["Porcentaje de Registros"].reset_index().sort_values(by='Indicator Code', ascending=False)

countries = gender['Country Name'].drop_duplicates().to_list()
for country in countries:
    plt.figure(figsize=(20, 10))
    sns.barplot(x="Indicator Code", y="Porcentaje de Registros", data=GenderFeaturesByID[GenderFeaturesByID["Country Name"]== country], color = "r")
    plt.xticks(fontsize=14, rotation=90)
    plt.yticks(fontsize=14)
    plt.xlabel('Indicator Code', fontsize=14)
    plt.ylabel('Porcentaje con Datos', fontsize=14)
    plt.title("Porcentaje con Datos por Indicador de " + str(country) + "\n", fontsize=18)
    plt.axhline(y=50,linewidth=1, color='black')


Como puede observarse en los gráficos de barra, la mayoría de los países tienen más del 50% de información de los de los features seleccionados para el análisis. Brasil es el país con mayor cantidad de features (13) que tienen un registro menor al 50%. Luego, le siguen Bolivia, Uruguay y Guyana.

In [ ]:
#Se seleccionan del dataset solo aquellas columnas relevantes para el gráfico
GenderFeaturesByID = RelevantFeaturesGender.drop(["Country Code", "Indicator", "features"], axis=1)
#Contamos todas las combinaciones existentes por país e indicator code para cada año
GenderFeaturesByID = GenderFeaturesByID.groupby(["Country Name", "Indicator Code"]).count()
#Sumamos las combinaciones de país e indicator code para todos los años, 
#Para luego, dado que la cantidad de años de análisis es 50, 
#obtener el porcentaje de registros por país e indicator code para todo el período en análisis
GenderFeaturesByID["Porcentaje de Registros"] = (GenderFeaturesByID.sum(axis=1)/50)*100
#Para Facilitar la lectura del cuadro, se crea una columna que identifica si el % de registros es mayor o menor a 50%
GenderFeaturesByID["Mayor a 50%"] = GenderFeaturesByID["Porcentaje de Registros"]>50
#Ordenamos por Indicator Code, para seleccionar las mismas variables para todos los países, y, luego, en caso que algún
# país no tenga más del 50% para algún feature, repetimos el proceso ordenando en base al % de registros.
GenderFeaturesByID = GenderFeaturesByID[["Porcentaje de Registros" , "Mayor a 50%"]].reset_index().sort_values(by='Indicator Code', ascending=False)
countries = gender['Country Name'].drop_duplicates()

GenderIDbyCountry = []
for country in countries:
       GenderIDbyCountry.append( GenderFeaturesByID.loc[GenderFeaturesByID["Country Name"]==country].iloc[0:15] )
GenderIDbyCountry

De los 15 features seleccionados para Género, todos los países cumplen con la condición de tener mas del 50% de datos en el período.

In [ ]:
#Se definen los features seleccionados

GenderIDbyCountry = GenderIDbyCountry.copy()
GenderIDFeatures = GenderIDbyCountry[0]["Indicator Code"].unique()
GenderIDFeatures

In [ ]:
                                                                                                 
print("Asignación de ID Code elegidos con su respectivo feature." )
gender[gender["Indicator Code"].isin(GenderIDFeatures)][["Indicator Code", "features"]].drop_duplicates().reset_index().drop(columns=["index"])

#### Funciones de Densidad por Feature para cada país

Para realizar los siguientes análisis, se construye un dataset que tenga solo las columnas relevantes de análisis y que solo considere los ID Features seleccionados en el paso anterior. 

In [ ]:
columns=gender.columns
GenderData=gender.drop(["Indicator", "Country Code", "features"], axis=1).rename(columns = lambda x : str(x)[:4] if x in columns[5:] else x)
GenderData=GenderData[GenderData["Indicator Code"].isin(GenderIDFeatures)]

In [ ]:
GenderDataMelt = pd.melt(GenderData, id_vars = ["Indicator Code" , "Country Name"])
GenderDataMelt["variable"] = pd.to_numeric(GenderDataMelt["variable"] )
GenderDataMelt = GenderDataMelt.rename(columns = {"variable": "Year"})
GenderDataMelt

Se crea un dataset por país

In [ ]:
GenderDataMelt = GenderDataMelt.copy()
arg=GenderDataMelt[GenderDataMelt["Country Name"]=="Argentina"]
bra=GenderDataMelt[GenderDataMelt["Country Name"]=="Brasil"]
ur=GenderDataMelt[GenderDataMelt["Country Name"]=="Uruguay"]
par=GenderDataMelt[GenderDataMelt["Country Name"]=="Paraguay"]
ven=GenderDataMelt[GenderDataMelt["Country Name"]=="Venezuela"]
bol=GenderDataMelt[GenderDataMelt["Country Name"]=="Bolivia"]
chi=GenderDataMelt[GenderDataMelt["Country Name"]=="Chile"]
guy=GenderDataMelt[GenderDataMelt["Country Name"]=="Guyana"]
sur=GenderDataMelt[GenderDataMelt["Country Name"]=="Suriname"]
col=GenderDataMelt[GenderDataMelt["Country Name"]=="Colombia"]
ecu=GenderDataMelt[GenderDataMelt["Country Name"]=="Ecuador"]
per=GenderDataMelt[GenderDataMelt["Country Name"]=="Perú"]


Se grafican las curvas de densidad para cada ID Features

In [ ]:
GenderIDbyCountry = GenderIDbyCountry.copy()
GenderIDFeatures = GenderIDbyCountry[0]["Indicator Code"].unique()

for Features in GenderIDFeatures:
    plt.figure(figsize=(13,6))
    sns.distplot(arg[arg["Indicator Code"]==Features]["value"], label="arg", hist=False)
    sns.distplot(bra[bra["Indicator Code"]==Features]["value"], label="bra", hist=False)
    sns.distplot(ur[ur["Indicator Code"]==Features]["value"], label="ur", hist=False)
    sns.distplot(par[par["Indicator Code"]==Features]["value"], label="par", hist=False)
    sns.distplot(ven[ven["Indicator Code"]==Features]["value"], label="ven", hist=False)
    sns.distplot(bol[bol["Indicator Code"]==Features]["value"], label="bol", hist=False)
    sns.distplot(chi[chi["Indicator Code"]==Features]["value"], label="chi", hist=False)
    sns.distplot(guy[guy["Indicator Code"]==Features]["value"], label="guy", hist=False)
    sns.distplot(sur[sur["Indicator Code"]==Features]["value"], label="sur", hist=False)
    sns.distplot(col[col["Indicator Code"]==Features]["value"], label="col", hist=False)
    sns.distplot(ecu[ecu["Indicator Code"]==Features]["value"], label="ecu", hist=False)
    sns.distplot(per[per["Indicator Code"]==Features]["value"], label="per", hist=False)
    plt.title("Funciones de densidad "+ str(Features) + " por país", fontsize=20)
    plt.legend(loc='upper center', bbox_to_anchor=(1.4, 1), shadow=True, ncol=2, fontsize= 'xx-small')
    plt.xticks(fontsize=16, rotation=90)
    plt.yticks(fontsize=16)
    plt.xlabel('Valor', fontsize=16)
   
  

    

Como puede evidenciarse en todos los histogramas, las distribuciones de cada feature en los diferentes países difiere bastante. Es decir, en la región cada uno de los features se distribuye diferente dependiendo del país que se analice. Es posible que con un análisis mas detallado de cada feature, se encuentren países donde la feature se distribuye similar.

#### BoxPlot por Feature

Para profundizar el análisis llevado a cabo hasta ahora, se construyen Box Plot para cada feature por país. 

In [ ]:
GenderIDbyCountry = GenderIDbyCountry.copy()
GenderIDFeatures = GenderIDbyCountry[0]["Indicator Code"].unique()
GenderDataMelt = GenderDataMelt.copy()

for Features in GenderIDFeatures:
    plt.figure(figsize=(13,6))
    sns.boxplot(data=GenderDataMelt[GenderDataMelt["Indicator Code"]==Features], x="Country Name", y="value")
    plt.title("Boxplot del feature "+ str(Features) + " por país", fontsize=24)
    plt.xticks(fontsize=16, rotation=90)
    plt.yticks(fontsize=16)
    plt.xlabel('País', fontsize=16)
    plt.ylabel('Valor', fontsize=16)

En lineas generales podria decirse que hay pocas features que tienen valores extremos (outliers).
Tambien a simple vista pareciera que la mayoria de las cajas son pequeñas para muchos de los features, esto habla de que los datos estan concentrados en torno a su mediana. En el caso de los países con cajas más grandes, tienden a ser los mismos para los distintos features; como es el caso de Bolivia, Brasil, Chile, Perú. 

#### Series Temporales de los Features por País

In [ ]:
countries = gender['Country Name'].drop_duplicates().to_list()
GenderDataMelt = GenderDataMelt.copy()
for country in countries:
    plt.figure(figsize=(10,6))
    sns.lineplot(data=GenderDataMelt[GenderDataMelt["Country Name"]==country], x="Year", y="value", hue="Indicator Code")
    plt.title("Serie Temporal de los Indicadores en " + str(country), fontsize=24)
    plt.xticks(fontsize=16, rotation=90)
    plt.yticks(fontsize=16)
    plt.xlabel('País', fontsize=16)
    plt.ylabel('Valor', fontsize=16)
    plt.legend(loc='upper center', bbox_to_anchor=(2, 1), shadow=True, ncol=2, fontsize= 'xx-small')

En términos generales, puede observarse que en todos los países en ánálisis, la Tasa de mortalidad, adultos, mujeres (por cada 1.000 mujeres adultas) (SP.DYN.AMRT.FE) y la Tasa de mortalidad, adultos, varones (por cada 1.000 varones adultos) (SP.DYN.AMRT.MA ), muestran una tendencia a la baja.


Llama la atención el comportamiento de la tasa de mortalidad masculina en Brasil, que muestra un aumento hasta el año 1987 y luego decrece.
A su vez, la tasa de mortalidad tanto masculina como femenina en Guyana muestra una tendencia a la baja, sin embargo, en los últimos años aumentó. Venezuela muestra una situación similar a Guyana. 


##### Gráficos Particulares: Casos Especiales

###### Brasil

In [ ]:
_condicion = (GenderDataMelt["Country Name"]=="Brasil") & (GenderDataMelt["Indicator Code"]=="SP.DYN.AMRT.MA")
plt.figure(figsize=(10,7))
sns.lineplot(data=GenderDataMelt[_condicion] , x="Year", y="value")
plt.title("Serie Temporal de la Tasa de Mortalidad Masculina en Brasil", fontsize=18)
sns.despine()
plt.xticks(fontsize=14, rotation=90)
plt.yticks(fontsize=14)
plt.xlabel('Año', fontsize=14)
plt.ylabel('Valor', fontsize=14)
plt.xticks(rotation=90)


In [ ]:
_condicion = (GenderDataMelt["Country Name"]=="Brasil") & (GenderDataMelt["Indicator Code"]=="SP.DYN.AMRT.MA")
_df = GenderDataMelt[_condicion]
_min = float(GenderDataMelt[_condicion]["value"].min())
_year_min = _df[_df["value"] == _min]
_year_min = int(_year_min["Year"])
print( "El valor mínimo que asume la variable es " + str(_min) + " en el año " + str(_year_min) + ".")
_max = float(GenderDataMelt[_condicion]["value"].max())
_year_max = _df[_df["value"] == _max]
_year_max = int(_year_max["Year"])
print( "El valor máximo que asume la variable es " + str(_max) + " en el año " + str(_year_max) + ".")
print( "*Vale destacar que la tasa de mortalidad es calculada cada 1000 personas.")

###### Guyana

In [ ]:
_condicion = (GenderDataMelt["Country Name"]=="Guyana") & ((GenderDataMelt["Indicator Code"]=="SP.DYN.AMRT.MA") | (GenderDataMelt["Indicator Code"]=="SP.DYN.AMRT.FE"))
plt.figure(figsize=(10,7))
sns.lineplot(data=GenderDataMelt[_condicion] , x="Year", y="value", hue = "Indicator Code")
plt.title("Serie Temporal de la Tasa de Mortalidad (MA y FE) en Guyana", fontsize=18)
sns.despine()
plt.xticks(fontsize=14, rotation=90)
plt.yticks(fontsize=14)
plt.xlabel('Año', fontsize=14)
plt.ylabel('Valor', fontsize=14)
plt.xticks(rotation=90)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)


In [ ]:
_condicion = ((GenderDataMelt["Country Name"]=="Guyana") & ((GenderDataMelt["Indicator Code"]=="SP.DYN.AMRT.MA") | (GenderDataMelt["Indicator Code"]=="SP.DYN.AMRT.FE")))
_df = GenderDataMelt[_condicion]
_features = ["SP.DYN.AMRT.MA", "SP.DYN.AMRT.FE"]

for Feature in _features:
          _min = float(_df[_df["Indicator Code"] == Feature]["value"].min())
          _year_min = _df[(_df["Indicator Code"] == Feature) & (_df["value"] == _min)]
          _year_min = int(_year_min["Year"])
          print( "El valor mínimo que asume la variable " + str(Feature) + " es " + str(_min) + " en el año " + str(_year_min) + ".")
          _max = float(_df[_df["Indicator Code"] == Feature]["value"].max())
          _year_max = _df[(_df["Indicator Code"] == Feature) & (_df["value"] == _max)]
          _year_max = int(_year_max["Year"])
          print( "El valor máximo que asume la variable " + str(Feature) + " es " + str(_max) + " en el año " + str(_year_max) + ".")


###### Venezuela

In [ ]:
_condicion = (GenderDataMelt["Country Name"]=="Venezuela") & ((GenderDataMelt["Indicator Code"]=="SP.DYN.AMRT.MA") | (GenderDataMelt["Indicator Code"]=="SP.DYN.AMRT.FE"))
plt.figure(figsize=(10,7))
sns.lineplot(data=GenderDataMelt[_condicion] , x="Year", y="value", hue = "Indicator Code")
plt.title("Serie Temporal de la Tasa de Mortalidad (MA y FE) en Venezuela", fontsize=18)
sns.despine()
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.xticks(fontsize=14, rotation=90)
plt.yticks(fontsize=14)
plt.xlabel('Año', fontsize=14)
plt.ylabel('Valor', fontsize=14)
plt.xticks(rotation=90)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

In [ ]:
_condicion = (GenderDataMelt["Country Name"]=="Venezuela") & ((GenderDataMelt["Indicator Code"]=="SP.DYN.AMRT.MA") | (GenderDataMelt["Indicator Code"]=="SP.DYN.AMRT.FE"))
_df = GenderDataMelt[_condicion]
_features = ["SP.DYN.AMRT.MA", "SP.DYN.AMRT.FE"]

for Feature in _features:
          _min = float(_df[_df["Indicator Code"] == Feature]["value"].min())
          _year_min = _df[(_df["Indicator Code"] == Feature) & (_df["value"] == _min)]
          _year_min = int(_year_min["Year"])
          print( "El valor mínimo que asume la variable " + str(Feature) + " es " + str(_min) + " en el año " + str(_year_min) + ".")
          _max = float(_df[_df["Indicator Code"] == Feature]["value"].max())
          _year_max = _df[(_df["Indicator Code"] == Feature) & (_df["value"] == _max)]
          _year_max = int(_year_max["Year"])
          print( "El valor máximo que asume la variable " + str(Feature) + " es " + str(_max) + " en el año " + str(_year_max) + ".")

### Determinación de Año a Predecir

Una vez seleccionados los indicadores a predecir por país para la dimensión de género, se verifica que todos los indicadores seleccionados tengan información para el año 2019, para elegirlo como año prueba. Si no se cumple la condición, entonces, se replica el análisis para el año 2018, y se opta por considerarlo como año a predecir.

In [ ]:
featureWOData_2019  = GenderData["2019"].isna().sum()
Data_2019 = (featureWOData_2019/len(GenderData["2019"]))*100

print("De las 15 features seleccionadas para género, el " + str(round(Data_2019, 2)) + "% no posee información para el año 2019." )

featureWOData_2018  = GenderData["2018"].isna().sum()

Data_2018 = (featureWOData_2018/len(GenderData["2018"]))*100

print("De las 15 features seleccionadas para género, el " + str(Data_2018) + "% no posee información para el año 2018." )



Se llega a la conclusión que no existe suficiente información para el año 2019, por eso se selecciona como año a predecir el 2018.

##SALUD


1. ¿Cuántos **tipos de Features diferentes** existen por indicador considerado? ¿Todos los países mantienen esa misma cantidad para el mismo indicador?

In [ ]:
print("El indicador de salud tiene " + str(len(health["features"].unique())) + " features.")
print("El análisis se lleva a cabo consdierando " + str(len(health["Country Name"].unique())) + " países.")
print(health["Country Name"].unique())

In [ ]:
countries = health['Country Name'].unique()

for countries in countries:
    
    print("La cantidad de features que posee " + str(countries) + ": " + str(health[health['Country Name']==countries]['features'].count()) )

### Análisis por Feature para toda la Región

2.  ¿Cuál es la **proporción de valores nulos** para cada Feature dentro de cada Indicador con respecto al intervalo temporal considerado? ¿Estos valores nulos se encuentran dispersos a lo largo de todo el intervalo temporal o existen agrupamientos (más chicos y/o más grandes) de los mismos? ¿Qué decisión se podría tomar con respecto a los mismos en un futuro?
3. ¿Cómo es la distribución de valores nulos por año (por columnas del dataset)?. Teniendo en cuenta que el objetivo final de la mentoría es realizar una predicción de cada Feature para el año 2019, ¿se disponen de datos suficientes para comparar las predicciones que se obtendrán con el valor real en ese año?


In [ ]:
print("Los features de salud que existen son los siguientes: \n", str(health['features'].unique()))

Para llevar a cabo el análisis de los datos nulos, se construye una matriz por feature y por año en la que se muestra el porcentaje de registros existentes para cada combinación considerando toda la región.

In [ ]:
HealthValuesByICode = health.groupby("Indicator Code").count().drop(["Country Code", "Country Name", "Indicator", "features"], axis=1)
HealthValuesByICode["Valor"] = HealthValuesByICode.sum(axis=1)
HealthValuesByICode = HealthValuesByICode.sort_values(by=['Valor'], ascending=False)
HealthValuesByICode = ((HealthValuesByICode/12)*100).drop(columns = ["Valor"])
HealthValuesByICode = HealthValuesByICode.rename(columns = lambda x : str(x)[:4])
round(HealthValuesByICode,2)

In [ ]:
plt.figure(figsize=(60, 30))
sns.heatmap(HealthValuesByICode, cbar=True, linewidths=0.1,  vmin=0, vmax=100)
sns.set(font_scale=4)

Este Indicador contiene la mayoria de sus datos completos, con muy pocos año con datos null.

In [ ]:
HealthValuesByICode = HealthValuesByICode.reset_index()
HealthValuesByICode["features"] = HealthValuesByICode["Indicator Code"]
HealthValuesByICode["features"] = HealthValuesByICode["features"].map(health.set_index("Indicator Code")["features"].to_dict())
FeaturesHealthSelection = HealthValuesByICode.iloc[0:90][["Indicator Code", "features"]].set_index("Indicator Code")
FeaturesHealthSelection

### Análisis por País para cada **Feature**

4. Volviendo a los resultados del punto 2, elegir como mínimo 15 Features por Indicador con al menos el 50% de valores no nulos (y en lo posible los mismos para todos los países) y mostrar gráficamente, con gráficos de distribución o de cajas por ejemplo, la distribución de los mismos. ¿Qué se puede observar de estas distribuciones? ¿Son normales? ¿Qué sucede con los outliers? ¿Observan alguna similitud entre los features elegidos?. Para estos mismos 15 Features realizar el gráfico de línea de su serie temporal y analizarlo.

In [ ]:
HealthIDCodeSelection = HealthValuesByICode.iloc[0:30].reset_index()
HealthIDCodeSelection = HealthIDCodeSelection["Indicator Code"].to_list()
HealthIDCodeSelection

In [ ]:
RelevantFeaturesHealth = health[health["Indicator Code"].isin(HealthIDCodeSelection)]
HealthDataByCountry = RelevantFeaturesHealth.groupby("Country Name").count().drop(["features", "Indicator", "Country Code", "Indicator Code"], axis=1)
HealthDataByCountry["Valor"] = HealthDataByCountry.sum(axis=1)
HealthDataByCountry = HealthDataByCountry.sort_values(by=['Valor'], ascending=False)
HealthDataByCountry = ((HealthDataByCountry/len(RelevantFeaturesHealth["Indicator Code"].unique()))*100).drop(columns = ["Valor"])
HealthDataByCountry = HealthDataByCountry.rename(columns = lambda x : str(x)[:4])
round(HealthDataByCountry,2)

In [ ]:
plt.figure(figsize=(40, 20))
sns.heatmap(HealthDataByCountry, cbar=True, linewidths=.3, vmin=0, vmax=100)
sns.set(font_scale=4)

Podemos observar que todos los paises tienen buena informacion, salvo para el año 2019.

Se realiza un heatmap considerando como centro el 50% de esta forma se puede visualizar cuáles son las combinaciones que tienen mas del 50% de registros con datos.

In [ ]:
plt.figure(figsize=(40, 20))
sns.heatmap(HealthDataByCountry, cbar=True, linewidths=.3, center=50, cmap="PRGn")
sns.set(font_scale=4)

In [ ]:
_porc_paises_mas_50_health = round((HealthDataByCountry[HealthDataByCountry>50].count()/len(health["Country Name"].unique()))*100, 0).astype(int)

print("El % de países con más del 50% de los features con datos para cada año es: \n" + str(_porc_paises_mas_50_health))

Se modifica la estructura del dataset para poder analizar el porcentaje de registros con datos por país, por feature y por año, para decidir qué features considerar para cada país.

In [ ]:
HealthRelevantFeaturesByID = RelevantFeaturesHealth.drop(["Country Code", "Indicator", "features"], axis=1)
HealthRelevantFeaturesByID = HealthRelevantFeaturesByID.groupby(["Country Name", "Indicator Code"]).count()
HealthRelevantFeaturesByID["Porcentaje de Registros"] = (HealthRelevantFeaturesByID.sum(axis=1)/50)*100
HealthRelevantFeaturesByID = HealthRelevantFeaturesByID["Porcentaje de Registros"].reset_index().sort_values(by='Indicator Code', ascending=False)

countries = health['Country Name'].drop_duplicates().to_list()
for countries in countries:
    plt.figure(figsize=(20, 10))
    sns.barplot(x="Indicator Code", y="Porcentaje de Registros", data=HealthRelevantFeaturesByID[HealthRelevantFeaturesByID["Country Name"]== countries], color = "r")
    plt.xticks(fontsize=14, rotation=90)
    plt.yticks(fontsize=14)
    plt.xlabel('Indicator Code', fontsize=14)
    plt.ylabel('Porcentaje con Datos', fontsize=14)
    plt.title("Porcentaje con Datos por Indicador de " + str(countries) + "\n", fontsize=16)
    plt.axhline(y=50,linewidth=1, color='black')

In [ ]:
#Se seleccionan del dataset solo aquellas columnas relevantes para el gráfico
HealthRelevantFeaturesByID = RelevantFeaturesHealth.drop(["Country Code", "Indicator", "features"], axis=1)
#Contamos todas las combinaciones existentes por país e indicator code para cada año
HealthRelevantFeaturesByID = HealthRelevantFeaturesByID.groupby(["Country Name", "Indicator Code"]).count()
#Sumamos las combinaciones de país e indicator code para todos los años, 
#Para luego, dado que la cantidad de años de análisis es 50, 
#obtener el porcentaje de registros por país e indicator code para todo el período en análisis
HealthRelevantFeaturesByID["Porcentaje de Registros"] = (HealthRelevantFeaturesByID.sum(axis=1)/50)*100
#Para Facilitar la lectura del cuadro, se crea una columna que identifica si el % de registros es mayor o menor a 50%
HealthRelevantFeaturesByID["Mayor a 50%"] = HealthRelevantFeaturesByID["Porcentaje de Registros"]>50
#Ordenamos por Indicator Code, para seleccionar las mismas variables para todos los países, y, luego, en caso que algún
# país no tenga más del 50% para algún feature, repetimos el proceso ordenando en base al % de registros.
HealthRelevantFeaturesByID = HealthRelevantFeaturesByID[["Porcentaje de Registros" , "Mayor a 50%"]].reset_index().sort_values(by='Indicator Code', ascending=False)
countries = health['Country Name'].drop_duplicates()

HealthIDbyCountry = []
for countries in countries:
       HealthIDbyCountry.append( HealthRelevantFeaturesByID.loc[HealthRelevantFeaturesByID["Country Name"]==countries].iloc[0:15] )
HealthIDbyCountry

De los 15 features seleccionados para Salud, todos los países cumplen con la condición de tener mas del 50% de datos en el período.

In [ ]:
#Se definen los features seleccionados

HealthIDFeatures = HealthIDbyCountry[0]["Indicator Code"].unique()
HealthIDFeatures

In [ ]:
print("Asignación de ID Code elegidos con su respectivo feature." )
health[health["Indicator Code"].isin(HealthIDFeatures)][["Indicator Code", "features"]].drop_duplicates().reset_index().drop(columns=["index"])

#### Funciones de Densidad por Feature para cada país

Para realizar los siguientes análisis, se construye un dataset que tenga solo las columnas relevantes de análisis y que solo considere los ID Features seleccionados en el paso anterior.

In [ ]:
columns=health.columns
HealthData=health.drop(["Indicator", "Country Code"], axis=1).rename(columns = lambda x : str(x)[:4] if x in columns[5:] else x)
HealthData=HealthData[HealthData["Indicator Code"].isin(HealthIDFeatures)]
HealthData

In [ ]:
HealthDataMelt = pd.melt(HealthData, id_vars = ["features" ,"Indicator Code" , "Country Name"])
HealthDataMelt["variable"] = pd.to_numeric(HealthDataMelt["variable"] )
HealthDataMelt = HealthDataMelt.rename(columns = {"variable": "Year"})

In [ ]:
arg_hea=HealthDataMelt[HealthDataMelt["Country Name"]=="Argentina"]
bra_hea=HealthDataMelt[HealthDataMelt["Country Name"]=="Brasil"]
ur_hea=HealthDataMelt[HealthDataMelt["Country Name"]=="Uruguay"]
par_hea=HealthDataMelt[HealthDataMelt["Country Name"]=="Paraguay"]
ven_hea=HealthDataMelt[HealthDataMelt["Country Name"]=="Venezuela"]
bol_hea=HealthDataMelt[HealthDataMelt["Country Name"]=="Bolivia"]
chi_hea=HealthDataMelt[HealthDataMelt["Country Name"]=="Chile"]
guy_hea=HealthDataMelt[HealthDataMelt["Country Name"]=="Guyana"]
sur_hea=HealthDataMelt[HealthDataMelt["Country Name"]=="Suriname"]
col_hea=HealthDataMelt[HealthDataMelt["Country Name"]=="Colombia"]
ecu_hea=HealthDataMelt[HealthDataMelt["Country Name"]=="Ecuador"]
per_hea=HealthDataMelt[HealthDataMelt["Country Name"]=="Perú"]

In [ ]:
HealthIDbyCountry = HealthIDbyCountry.copy()
HealthIDFeatures = HealthIDbyCountry[0]["Indicator Code"].unique()

for HealthIDFeatures in HealthIDFeatures:
    plt.figure(figsize=(13,6))
    sns.distplot(arg_hea[arg_hea["Indicator Code"]==HealthIDFeatures]["value"], label="arg", hist=False)
    sns.distplot(bra_hea[bra_hea["Indicator Code"]==HealthIDFeatures]["value"], label="bra", hist=False)
    sns.distplot(ur_hea[ur_hea["Indicator Code"]==HealthIDFeatures]["value"], label="ur", hist=False)
    sns.distplot(par_hea[par_hea["Indicator Code"]==HealthIDFeatures]["value"], label="par", hist=False)
    sns.distplot(ven_hea[ven_hea["Indicator Code"]==HealthIDFeatures]["value"], label="ven", hist=False)
    sns.distplot(bol_hea[bol_hea["Indicator Code"]==HealthIDFeatures]["value"], label="bol", hist=False)
    sns.distplot(chi_hea[chi_hea["Indicator Code"]==HealthIDFeatures]["value"], label="chi", hist=False)
    sns.distplot(guy_hea[guy_hea["Indicator Code"]==HealthIDFeatures]["value"], label="guy", hist=False)
    sns.distplot(sur_hea[sur_hea["Indicator Code"]==HealthIDFeatures]["value"], label="sur", hist=False)
    sns.distplot(col_hea[col_hea["Indicator Code"]==HealthIDFeatures]["value"], label="col", hist=False)
    sns.distplot(ecu_hea[ecu_hea["Indicator Code"]==HealthIDFeatures]["value"], label="ecu", hist=False)
    sns.distplot(per_hea[per_hea["Indicator Code"]==HealthIDFeatures]["value"], label="per", hist=False)
    plt.title("Funciones de densidad "+ str(Features) + " por país", fontsize=20)
    plt.legend(loc='upper center', bbox_to_anchor=(1.4, 1), shadow=True, ncol=2, fontsize= 'xx-small')
    plt.xticks(fontsize=16, rotation=90)
    plt.yticks(fontsize=16)
    plt.xlabel('Valor', fontsize=16)

    

De los graficos anteriores se desprende que existe una gran heterogeneidad entre países. Para muchos de los features se observa como Bolivia y Argentina presentan una elevada curtosis en relación al resto de los países. 

#### BoxPlot por Feature

Para profundizar el análisis llevado a cabo hasta ahora, se construyen Box Plot para cada feature por país. 

In [ ]:
HealthIDbyCountry = HealthIDbyCountry.copy()
HealthIDFeatures = HealthIDbyCountry[0]["Indicator Code"].unique()
HealthDataMelt = HealthDataMelt.copy()

for Features in HealthIDFeatures:
    plt.figure(figsize=(13,6))
    sns.boxplot(data=HealthDataMelt[HealthDataMelt["Indicator Code"]==Features], x="Country Name", y="value")
    plt.legend(loc='upper center', bbox_to_anchor=(1.4, 1), shadow=True)
    plt.xticks(fontsize=16, rotation=90)
    plt.yticks(fontsize=16)
    plt.xlabel('País', fontsize=16)
    plt.ylabel('Valor', fontsize=16)


Los boxplot permiten observar la existencia de valores atipicos para distintos países en cada uno de los features. Los tamaños de las cajas son variados por feauture y por país, en términos generales los países con menor rango de datos son Argentina y Bolivia, que presentan cajas pequeñas. El resto de los países presentan cajas disimiles para los diferentes features.

#### Series Temporales de los Features por País

In [ ]:
countries = health['Country Name'].drop_duplicates().to_list()
HealthDataMelt = GenderDataMelt.copy()
for country in countries:
    plt.figure(figsize=(10,7))
    sns.lineplot(data=HealthDataMelt[HealthDataMelt["Country Name"]==country], x="Year", y="value", hue="Indicator Code")
    plt.title("Serie Temporal de los Indicadores en " + str(country), fontsize=18)
    sns.despine()
    plt.xticks(fontsize=14, rotation=90)
    plt.yticks(fontsize=14)
    plt.xlabel('País', fontsize=14)
    plt.ylabel('Valor', fontsize=14)
    plt.xticks(rotation=90)
    plt.legend(loc='upper center', bbox_to_anchor=(2, 1), shadow=True, ncol=2, fontsize= 'xx-small')

### Determinación del Año a Predecir

Una vez seleccionados los indicadores a predecir por país para la dimensión de género, se verifica que todos los indicadores seleccionados tengan información para el año 2019, para elegirlo como año prueba. Si no se cumple la condición, entonces, se replica el análisis para el año 2018, y se opta por considerarlo como año a predecir.

In [ ]:
featureWOData_2019  = HealthData["2019"].isna().sum()
Data_2019 = (featureWOData_2019/len(HealthData["2019"]))*100

print("De las 15 features seleccionadas para salud, el " + str(round(Data_2019, 2)) + "% no posee información para el año 2019." )

featureWOData_2018  = HealthData["2018"].isna().sum()

Data_2018 = (featureWOData_2018/len(HealthData["2018"]))*100

print("De las 15 features seleccionadas para salud, el " + str(Data_2018) + "% no posee información para el año 2018." )


Consecuentemente, se proyectará el año 2019.

## EDUCACIÓN

1. ¿Cuántos **tipos de Features diferentes** existen por indicador considerado? ¿Todos los países mantienen esa misma cantidad para el mismo indicador?

Existen 106 features diferentes por país para el indicador educación. Todos los países poseen la misma cantidad.

In [ ]:
print("El indicador de educación tiene " + str(len(education["features"].unique())) + " features.")
print("El análisis se lleva a cabo consdierando " + str(len(education["Country Name"].unique())) + " países.")
print(education["Country Name"].unique())

In [ ]:
countries = education['Country Name'].unique()

for countries in countries:
    
    print("La cantidad de features que posee " + str(countries) + ": " + str(education[education['Country Name']==countries]['features'].count()) )

### Análisis por Feature para toda la Región

2. ¿Cuál es la proporción de valores nulos para cada Feature dentro de cada Indicador con respecto al intervalo temporal considerado? ¿Estos valores nulos se encuentran dispersos a lo largo de todo el intervalo temporal o existen agrupamientos (más chicos y/o más grandes) de los mismos? ¿Qué decisión se podría tomar con respecto a los mismos en un futuro?
3. ¿Cómo es la distribución de valores nulos por año (por columnas del dataset)?. Teniendo en cuenta que el objetivo final de la mentoría es realizar una predicción de cada Feature para el año 2019, ¿se disponen de datos suficientes para comparar las predicciones que se obtendrán con el valor real en ese año?

In [ ]:
print("Los features de educación que existen son los siguientes: \n", str(education['features'].unique()))

Para llevar a cabo el análisis de los datos nulos, se construye una matriz por feature y por año en la que se muestra el porcentaje de registros existentes para cada combinación considerando toda la región (promedio de todos los países).

In [ ]:
EducationValuesByICode = education.groupby("Indicator Code").count().drop(["Country Code", "Country Name", "Indicator", "features"], axis=1)
EducationValuesByICode["Valor"] = EducationValuesByICode.sum(axis=1)
EducationValuesByICode = EducationValuesByICode.sort_values(by=['Valor'], ascending=False)
EducationValuesByICode = ((EducationValuesByICode/12)*100).drop(columns = ["Valor"])
EducationValuesByICode = EducationValuesByICode.rename(columns = lambda x : str(x)[:4])
round(EducationValuesByICode,2)


Se construye un mapa de calor para visualizar aquellas features con mayor porcentaje de registros, mientras mas oscuro el color menos proporción de registros hay para dicha combinación.

In [ ]:
plt.figure(figsize=(60, 30))
sns.heatmap(EducationValuesByICode, cbar=True, linewidths=0.1,  vmin=0, vmax=100)
sns.set(font_scale=4)

Tomando el promedio de los países se observa como, para la mayoria de los features, hay valores nulos hasta 1997 y en 2018/2019. Donde exite menor proporción de valores nulos para la mayoría de los features es en el intervalo de tiempo desde 1998 a 2017. Por su parte, hay unos pocos features (los primeros del mapa de calor) que no siguen estas tendencias y poseen gran porporción de valores no nulos para la mayoria de los años.

In [ ]:
EducationByYear = pd.melt(education.drop(["Country Code", "Country Name", "Indicator", "features"], axis=1), id_vars = ["Indicator Code"])
EducationByYear["variable"] = pd.to_numeric(EducationByYear["variable"] )
EducationByYear = EducationByYear.rename(columns = {"variable": "Year"})
TotalData_1997 = (EducationByYear["Year"]<=1997).sum()
WithData_1997 = ((EducationByYear["Year"]<=1997) & (EducationByYear["value"]>0)).sum()

print( "El % de registros sin datos para el período de 1970 a 1997 es de: \n" +  str(round((1 - (WithData_1997/TotalData_1997))*100,2)) + "%") 

TotalData_2019 = (EducationByYear["Year"]>1997).sum()
WithData_2019 = ((EducationByYear["Year"]>1997) & (EducationByYear["value"]>0)).sum()
Check = len (EducationByYear) == (TotalData_2019 + TotalData_1997 )


print( "El % de registros sin datos para el período de 1991 a 2019 es de: \n" +  str(round((1 - (WithData_2019/TotalData_2019))*100,2)) + "%") 

### Análisis por País para cada Feature


Se seleccionan los primeros 15 features que poseen más datos:

In [ ]:
EducationValuesByICode=EducationValuesByICode.reset_index()
EducationValuesByICode["features"] = EducationValuesByICode["Indicator Code"]
EducationValuesByICode["features"] = EducationValuesByICode["features"].map(education.set_index("Indicator Code")["features"].to_dict())
FeaturesEducationSelection = EducationValuesByICode.iloc[0:15][["Indicator Code", "features"]].set_index("Indicator Code")
FeaturesEducationSelection

Se construye una matriz para determinar cuál es el porcentaje de registros con datos considerando las 15 features seleccionadas por país y año. 

In [ ]:
EducationIDCode = FeaturesEducationSelection.reset_index()['Indicator Code'].to_list()

RelevantFeaturesEducation = education[education["Indicator Code"].isin(EducationIDCode)]
EducationDataByCountry = RelevantFeaturesEducation.groupby("Country Name").count().drop(["features", "Indicator", "Country Code", "Indicator Code"], axis=1)
EducationDataByCountry["Valor"] = EducationDataByCountry.sum(axis=1)
EducationDataByCountry = EducationDataByCountry.sort_values(by=['Valor'], ascending=False)
EducationDataByCountry = ((EducationDataByCountry/len(RelevantFeaturesEducation["Indicator Code"].unique()))*100).drop(columns = ["Valor"])
EducationDataByCountry = EducationDataByCountry.rename(columns = lambda x : str(x)[:4])
round(EducationDataByCountry,2)

In [ ]:
plt.figure(figsize=(40, 20))
sns.heatmap(EducationDataByCountry, cbar=True, linewidths=.3, vmin=0, vmax=100)
sns.set(font_scale=4)

Se realiza un heatmap considerando como centro el 50% de esta forma se puede visualizar cuáles son las combinaciones que tienen mas del 50% de registros con datos.

In [ ]:
plt.figure(figsize=(40, 20))
sns.heatmap(EducationDataByCountry, cbar=True, linewidths=.3, center=50, cmap="PRGn")
sns.set(font_scale=4)

In [ ]:
_porc_paises_mas_50_edu= round((EducationDataByCountry[EducationDataByCountry>50].count()/len(education["Country Name"].unique()))*100, 0).astype(int)

print("El % de países que tienen más del 50% de los features con datos para cada año es: \n" + str(_porc_paises_mas_50_edu))

In [ ]:
EducationByYear = pd.melt(RelevantFeaturesEducation, id_vars = ["features", "Indicator" , "Country Name","Country Code", "Indicator Code"])
EducationByYear["variable"] = pd.to_numeric(EducationByYear["variable"] )
EducationByYear = EducationByYear.rename(columns = {"variable": "Year"})
EducationByYear["Year"] = EducationByYear["Year"].apply(lambda x: str(x)[:4])

EducationByYear

A partir de 1997, la mayoría de los paises en análisis tienen información para más del 50% de los features relevantes. Esto se evidencia en el heatmap, donde aquellas combinaciones con más del 50% de registros se muestran en diferentes tonalidades de verdes. Aquellas combinaciones que no alcanzan el 50% de los registros tienen diferentes tonalidades de violeta. El gráfico anticipa que para el caso de Brasil deberan contemplarse otros features puesto que pareciera presentar gran cantidad de nulos.

Se modifica la estructura del dataset para poder analizar el porcentaje de registros con datos por país, por feature y por año, para decidir qué features considerar para cada país.

In [ ]:
EducationRelevantFeaturesByID = RelevantFeaturesEducation.drop(["Country Code", "Indicator", "features"], axis=1)
EducationRelevantFeaturesByID = EducationRelevantFeaturesByID.groupby(["Country Name", "Indicator Code"]).count()
EducationRelevantFeaturesByID["Porcentaje de Registros"] = (EducationRelevantFeaturesByID.sum(axis=1)/50)*100
#RelevantFeaturesByID = RelevantFeaturesByID["Porcentaje de Registros"].reset_index().sort_values(by='Porcentaje de Registros', ascending=True)
EducationRelevantFeaturesByID = EducationRelevantFeaturesByID["Porcentaje de Registros"].reset_index().sort_values(by='Indicator Code', ascending=False)

countries = education['Country Name'].drop_duplicates().to_list()
for countries in countries:
    plt.figure(figsize=(20, 10))
    sns.barplot(x="Indicator Code", y="Porcentaje de Registros", data=EducationRelevantFeaturesByID[EducationRelevantFeaturesByID["Country Name"]== countries], color = "r")
    plt.xticks(fontsize=14, rotation=90)
    plt.yticks(fontsize=14)
    plt.xlabel('Indicator Code', fontsize=14)
    plt.ylabel('Porcentaje con Datos', fontsize=14)
    plt.title("Porcentaje con Datos por Indicador de " + str(countries) + "\n", fontsize=16)
    plt.axhline(y=50,linewidth=1, color='black')

In [ ]:
#Se seleccionan del dataset solo aquellas columnas relevantes para el gráfico
EducationRelevantFeaturesByID = RelevantFeaturesEducation.drop(["Country Code", "Indicator", "features"], axis=1)
#Contamos todas las combinaciones existentes por país e indicator code para cada año
EducationRelevantFeaturesByID = EducationRelevantFeaturesByID.groupby(["Country Name", "Indicator Code"]).count()
#Sumamos las combinaciones de país e indicator code para todos los años, 
#Para luego, dado que la cantidad de años de análisis es 50, 
#obtener el porcentaje de registros por país e indicator code para todo el período en análisis
EducationRelevantFeaturesByID["Porcentaje de Registros"] = (EducationRelevantFeaturesByID.sum(axis=1)/50)*100
#Para Facilitar la lectura del cuadro, se crea una columna que identifica si el % de registros es mayor o menor a 50%
EducationRelevantFeaturesByID["Mayor a 50%"] = EducationRelevantFeaturesByID["Porcentaje de Registros"]>50
#Ordenamos por Indicator Code, para seleccionar las mismas variables para todos los países, y, luego, en caso que algún
# país no tenga más del 50% para algún feature, repetimos el proceso ordenando en base al % de registros.
EducationRelevantFeaturesByID = EducationRelevantFeaturesByID[["Porcentaje de Registros" , "Mayor a 50%"]].reset_index().sort_values(by='Indicator Code', ascending=False)
countries = education['Country Name'].drop_duplicates()
EducationIDbyCountry = []
for countries in countries:
       EducationIDbyCountry.append( EducationRelevantFeaturesByID.loc[EducationRelevantFeaturesByID["Country Name"]==countries].iloc[0:15] )
EducationIDbyCountry

De aquí se desprende que para Guyana y Brasil deben cambiarse algunos features.

In [ ]:
# Buscamos primeros 15 features con mas datos para Brasil

BraEducationValuesByICode = education[education["Country Name"]=="Brasil"].groupby("Indicator Code").count().drop(["Country Code", "Indicator", "features"], axis=1)
BraEducationValuesByICode["Valor"] = BraEducationValuesByICode.sum(axis=1)
BraEducationValuesByICode = BraEducationValuesByICode.sort_values(by=['Valor'], ascending=False)
BraEducationValuesByICode = ((BraEducationValuesByICode/12)*100).drop(columns = ["Valor"])
BraEducationValuesByICode = BraEducationValuesByICode.rename(columns = lambda x : str(x)[:4])

BraEducationValuesByICode.reset_index().iloc[0:15]["Indicator Code"].to_list()

In [ ]:
# Buscamos primeros 15 features con mas datos para Brasil

BraEducationValuesByICode = education[education["Country Name"]=="Brasil"].groupby("Indicator Code").count().drop(["Country Code", "Indicator", "features"], axis=1)
BraEducationValuesByICode["Valor"] = BraEducationValuesByICode.sum(axis=1)
BraEducationValuesByICode = BraEducationValuesByICode.sort_values(by=['Valor'], ascending=False)
BraEducationValuesByICode = ((BraEducationValuesByICode/12)*100).drop(columns = ["Valor"])
BraEducationValuesByICode = BraEducationValuesByICode.rename(columns = lambda x : str(x)[:4])

BraEducationValuesByICode.reset_index().iloc[0:15]["Indicator Code"].to_list()

De lo anterior se decide añadir a la lista de features, dado que Brasil y Guyana disponen más datos al respecto, a los indicadores SE.TER.ENRL.TC.ZS, SE.PRE.ENRL.TC.ZS, SL.TLF.TOTL.IN, SL.TLF.TOTL.FE.ZS, SE.PRM.REPT.ZS, SL.UEM.TOTL.FE.ZS. Nos quedan, en total, 21 features.

In [ ]:
#Se definen los features seleccionados

EducationIDbyCountry = EducationIDbyCountry.copy()
EducationIDFeatures = EducationIDbyCountry[0]["Indicator Code"].to_list() + ["SE.TER.ENRL.TC.ZS", "SE.PRE.ENRL.TC.ZS", "SL.TLF.TOTL.IN", "SL.TLF.TOTL.FE.ZS", "SE.PRM.REPT.ZS", "SL.UEM.TOTL.FE.ZS"]
EducationIDFeatures

In [ ]:
print("Asignación de ID Code elegidos con su respectivo feature." )
education[education["Indicator Code"].isin(EducationIDFeatures)][["Indicator Code", "features"]].drop_duplicates().reset_index().drop(columns=["index"])

#### Funciones de Densidad por Feature para cada país



Para realizar los siguientes análisis, se construye un dataset que tenga solo las columnas relevantes de análisis y que solo considere los ID Features seleccionados en el paso anterior.

In [ ]:
columns=education.columns
EducationData=education.drop(["Indicator", "Country Code", "features"], axis=1).rename(columns = lambda x : str(x)[:4] if x in columns[5:] else x)
EducationData=EducationData[EducationData["Indicator Code"].isin(EducationIDFeatures)]
EducationData

In [ ]:
EducationDataMelt = pd.melt(EducationData, id_vars = ["Indicator Code" , "Country Name"])
EducationDataMelt["variable"] = pd.to_numeric(EducationDataMelt["variable"] )
EducationDataMelt = EducationDataMelt.rename(columns = {"variable": "Year"})
EducationDataMelt

Se crea un dataset para cada país:

In [ ]:
EducationDataMelt = EducationDataMelt.copy()
arg_ed=EducationDataMelt[EducationDataMelt["Country Name"]=="Argentina"]
bra_ed=EducationDataMelt[EducationDataMelt["Country Name"]=="Brasil"]
ur_ed=EducationDataMelt[EducationDataMelt["Country Name"]=="Uruguay"]
par_ed=EducationDataMelt[EducationDataMelt["Country Name"]=="Paraguay"]
ven_ed=EducationDataMelt[EducationDataMelt["Country Name"]=="Venezuela"]
bol_ed=EducationDataMelt[EducationDataMelt["Country Name"]=="Bolivia"]
chi_ed=EducationDataMelt[EducationDataMelt["Country Name"]=="Chile"]
guy_ed=EducationDataMelt[EducationDataMelt["Country Name"]=="Guyana"]
sur_ed=EducationDataMelt[EducationDataMelt["Country Name"]=="Suriname"]
col_ed=EducationDataMelt[EducationDataMelt["Country Name"]=="Colombia"]
ecu_ed=EducationDataMelt[EducationDataMelt["Country Name"]=="Ecuador"]
per_ed=EducationDataMelt[EducationDataMelt["Country Name"]=="Perú"]

Se grafican los histogramas para cada ID Features:

In [ ]:
EducationIDbyCountry = EducationIDbyCountry.copy()
EducationIDFeatures = EducationIDbyCountry[0]["Indicator Code"].unique()

for Features in EducationIDFeatures:
    plt.figure(figsize=(13,6))
    sns.distplot(arg_ed[arg_ed["Indicator Code"]==Features]["value"], label="arg", hist=False)
    sns.distplot(bra_ed[bra_ed["Indicator Code"]==Features]["value"], label="bra", hist=False)
    sns.distplot(ur_ed[ur_ed["Indicator Code"]==Features]["value"], label="ur", hist=False)
    sns.distplot(par_ed[par_ed["Indicator Code"]==Features]["value"], label="par", hist=False)
    sns.distplot(ven_ed[ven_ed["Indicator Code"]==Features]["value"], label="ven", hist=False)
    sns.distplot(bol_ed[bol_ed["Indicator Code"]==Features]["value"], label="bol", hist=False)
    sns.distplot(chi_ed[chi_ed["Indicator Code"]==Features]["value"], label="chi", hist=False)
    sns.distplot(guy_ed[guy_ed["Indicator Code"]==Features]["value"], label="guy", hist=False)
    sns.distplot(sur_ed[sur_ed["Indicator Code"]==Features]["value"], label="sur", hist=False)
    sns.distplot(col_ed[col_ed["Indicator Code"]==Features]["value"], label="col", hist=False)
    sns.distplot(ecu_ed[ecu_ed["Indicator Code"]==Features]["value"], label="ecu", hist=False)
    sns.distplot(per_ed[per_ed["Indicator Code"]==Features]["value"], label="per", hist=False)
    plt.title("Funciones de densidad "+ str(Features) + " por país", fontsize=20)
    plt.legend(loc='upper center', bbox_to_anchor=(1.4, 1), shadow=True, ncol=2, fontsize= 'xx-small')
    plt.xticks(fontsize=16, rotation=90)
    plt.yticks(fontsize=16)
    plt.xlabel('Valor', fontsize=16)
  

    

Del analisis de los histogramas por features para cada país se desprende que existe gran heterogeneidad entre países para muchos de los features considerados, presentando funciones de densidad diversas.

#### BoxPlot por Feature

Para profundizar el análisis llevado a cabo hasta ahora, se construyen Box Plot para cada feature por país.

In [ ]:
EducationIDbyCountry = EducationIDbyCountry.copy()
EducationIDFeatures = EducationIDbyCountry[0]["Indicator Code"].unique()
EducationDataMelt = EducationDataMelt.copy()

for Features in EducationIDFeatures:
    plt.figure(figsize=(13,6))
    sns.boxplot(data=EducationDataMelt[EducationDataMelt["Indicator Code"]==Features], x="Country Name", y="value")
    plt.legend(loc='upper center', bbox_to_anchor=(1.4, 1), shadow=True)
    plt.xticks(fontsize=16, rotation=90)
    plt.yticks(fontsize=16)
    plt.xlabel('País', fontsize=16)
    plt.ylabel('Valor', fontsize=16)
    plt.title("Boxplot del feature "+ str(Features) + " por país", fontsize=24)

Del analisis de los boxplot por indicador se vuelve a observar que existe mucha heterogeneidad entre países. La presencia de outliers para algunos features es practicamente nula, mientras que para otros es importante y ejerce presión sobre la media.

#### Series Temporales de los Features por País

In [ ]:
countries = education['Country Name'].drop_duplicates().to_list()

for country in countries:
    plt.figure(figsize=(10,7))
    sns.lineplot(data=EducationDataMelt[EducationDataMelt["Country Name"]==country], x="Year", y="value", hue="Indicator Code")
    plt.title("Serie Temporal de los Indicadores en " + str(country))
    sns.despine()
    plt.xticks(rotation=90)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

### Determinación de Año a Predecir

Una vez seleccionados los indicadores a predecir por país para la dimensión de educación, se verifica que todos los indicadores seleccionados tengan información para el año 2019, para elegirlo como año prueba. Si no se cumple la condición, entonces, se replica el análisis para el año 2018, y se opta por considerarlo como año a predecir.

In [ ]:
featureWOData_2019_Ed  = EducationData["2019"].isna().sum()
Data_2019_Ed = (featureWOData_2019_Ed/len(EducationData["2019"]))*100

print("De las 21 features seleccionadas para educación, el \n" + str(round(Data_2019_Ed, 2)) + "% no posee información para el año 2019." )

featureWOData_2018_Ed  = EducationData["2018"].isna().sum()

Data_2018_Ed = (featureWOData_2018/len(EducationData["2018"]))*100

print("De las 21 features seleccionadas para educación, el \n" + str(Data_2018_Ed) + "% no posee información para el año 2018." )



Por lo tanto, tomaremos el año 2018 como base.

## PROTECCIÓN SOCIAL

1. ¿Cuántos **tipos de Features diferentes** existen por indicador considerado? ¿Todos los países mantienen esa misma cantidad para el mismo indicador?

In [ ]:
print("El indicador de Protección Social tiene " + str(len(social_protection["features"].unique())) + " features.")
print("El análisis se lleva a cabo consdierando " + str(len(social_protection["Country Name"].unique())) + " países.")
print(social_protection["Country Name"].unique())

In [ ]:
countries = social_protection['Country Name'].unique()

for countries in countries:
    print("La cantidad de features que posee " + str(countries) + ": " + str(social_protection[social_protection['Country Name']==countries]['features'].count()))

### Análisis por Feature para toda la Región

2.  ¿Cuál es la **proporción de valores nulos** para cada Feature dentro de cada Indicador con respecto al intervalo temporal considerado? ¿Estos valores nulos se encuentran dispersos a lo largo de todo el intervalo temporal o existen agrupamientos (más chicos y/o más grandes) de los mismos? ¿Qué decisión se podría tomar con respecto a los mismos en un futuro?
3. ¿Cómo es la distribución de valores nulos por año (por columnas del dataset)?. Teniendo en cuenta que el objetivo final de la mentoría es realizar una predicción de cada Feature para el año 2019, ¿se disponen de datos suficientes para comparar las predicciones que se obtendrán con el valor real en ese año?

In [ ]:
print("Los features de Protección Social que existen son los siguientes: \n", str(social_protection['features'].unique()))

Para llevar a cabo el análisis de los datos nulos, se construye una matriz por feature y por año en la que se muestra el porcentaje de registros existentes para cada combinación considerando toda la región.

In [ ]:
SocProtValuesByICode = social_protection.groupby("Indicator Code").count().drop(["Country Code", "Country Name", "Indicator", "features"], axis=1)
SocProtValuesByICode["Valor"] = SocProtValuesByICode.sum(axis=1)
SocProtValuesByICode = SocProtValuesByICode.sort_values(by=['Valor'], ascending=False)
SocProtValuesByICode = ((SocProtValuesByICode/12)*100).drop(columns = ["Valor"])
SocProtValuesByICode = SocProtValuesByICode.rename(columns = lambda x : str(x)[:4])
round(SocProtValuesByICode,2)


Se construye un mapa de calor para visualizar aquellas features con mayor porcentaje de registros, mientras mas oscuro el color menos proporción de registros hay para dicha combinación.

In [ ]:
plt.figure(figsize=(60, 30))
sns.heatmap(SocProtValuesByICode, cbar=True, linewidths=0.1,  vmin=0, vmax=100)
sns.set(font_scale=4)

Como puede observarse graficamente, durante el periodo de 1970 a 1990 se concentra la mayor cantidad de nulos.

In [ ]:
SocProtByYear = pd.melt(social_protection.drop(["Country Code", "Country Name", "Indicator", "features"], axis=1), id_vars = ["Indicator Code"])
SocProtByYear["variable"] = pd.to_numeric(SocProtByYear["variable"] )
SocProtByYear = SocProtByYear.rename(columns = {"variable": "Year"})
TotalData_1990 = (SocProtByYear["Year"]<=1990).sum()
WithData_1990 = ((SocProtByYear["Year"]<=1990) & (SocProtByYear["value"]>0)).sum()

print( "El % de registros sin datos para el período de 1970 a 1990 es de: \n" +  str(round((1 - (WithData_1990/TotalData_1990))*100,2)) + "%") 

TotalData_2019 = (SocProtByYear["Year"]>1990).sum()
WithData_2019 = ((SocProtByYear["Year"]>1990) & (SocProtByYear["value"]>0)).sum()
Check = len (SocProtByYear) == (TotalData_2019 + TotalData_1990 )


print( "El % de registros sin datos para el período de 1991 a 2019 es de: \n" +  str(round((1 - (WithData_2019/TotalData_2019))*100,2)) + "%") 




Se seleccionan en un principio los 30 features con mas datos en toda la región:

In [ ]:
SocProtByYear = SocProtByYear.copy()

SocProtByYear = SocProtByYear.reset_index()
SocProtByYear["features"] = SocProtByYear["Indicator Code"]
SocProtByYear["features"] = SocProtByYear["features"].map(social_protection.set_index("Indicator Code")["features"].to_dict())
FeaturesSocProtSelection = SocProtByYear.iloc[0:30][["Indicator Code", "features"]].set_index("Indicator Code")
FeaturesSocProtSelection

### Análisis por País para cada Feature

4. Volviendo a los resultados del punto 2, elegir como mínimo 15 Features por Indicador con al menos el 50% de valores no nulos (y en lo posible los mismos para todos los países) y mostrar gráficamente, con gráficos de distribución o de cajas por ejemplo, la distribución de los mismos. ¿Qué se puede observar de estas distribuciones? ¿Son normales? ¿Qué sucede con los outliers? ¿Observan alguna similitud entre los features elegidos?. Para estos mismos 15 Features realizar el gráfico de línea de su serie temporal y analizarlo.

Se construye una matriz para determinar cuál es el porcentaje de registros con datos considerando las 32 features seleccionadas por país y año. Es decir, por ejemplo, para Perú en el año 1970 el 3.33% de los features tiene valores.

In [ ]:
SocProtIDCode = FeaturesSocProtSelection.reset_index()['Indicator Code'].to_list()

RelevantFeaturesSocProt = social_protection[social_protection["Indicator Code"].isin(SocProtIDCode)]
SocProtDataByCountry = RelevantFeaturesSocProt.groupby("Country Name").count().drop(["features", "Indicator", "Country Code", "Indicator Code"], axis=1)
SocProtDataByCountry["Valor"] = SocProtDataByCountry.sum(axis=1)
SocProtDataByCountry = SocProtDataByCountry.sort_values(by=['Valor'], ascending=False)
SocProtDataByCountry = ((SocProtDataByCountry/len(RelevantFeaturesSocProt["Indicator Code"].unique()))*100).drop(columns = ["Valor"])
SocProtDataByCountry = SocProtDataByCountry.rename(columns = lambda x : str(x)[:4])
round(SocProtDataByCountry,2)

In [ ]:
plt.figure(figsize=(40, 20))
sns.heatmap(SocProtDataByCountry, cbar=True, linewidths=.3, vmin=0, vmax=100)
sns.set(font_scale=4)

Se realiza un heatmap considerando como centro el 50% de esta forma se puede visualizar cuáles son las combinaciones que tienen mas del 50% de registros con datos.

In [ ]:
plt.figure(figsize=(40, 20))
sns.heatmap(SocProtDataByCountry, cbar=True, linewidths=.3, center=50, cmap="PRGn")
sns.set(font_scale=4)

In [ ]:
_porc_paises_mas_50_sp= round((SocProtDataByCountry[SocProtDataByCountry>50].count()/len(social_protection["Country Name"].unique()))*100, 0).astype(int)

print("El % de países con más del 50% de los features con datos para cada año es: \n" + str(_porc_paises_mas_50_sp))

In [ ]:
SocProtByYear = pd.melt(RelevantFeaturesSocProt, id_vars = ["features", "Indicator" , "Country Name","Country Code", "Indicator Code"])
SocProtByYear["variable"] = pd.to_numeric(SocProtByYear["variable"] )
SocProtByYear = SocProtByYear.rename(columns = {"variable": "Year"})
SocProtByYear["Year"] = SocProtByYear["Year"].apply(lambda x: str(x)[:4])

SocProtByYear

A partir de 1991, todos los paises en análisis tienen información para más del 50% de los features relevantes. Esto se evidencia en el heatmap, donde aquellas combinaciones con más del 50% de registros se muestran en diferentes tonalidades de verdes. Aquellas combinaciones que no alcanzan el 50% de los registros tienen diferentes tonalidades de violeta.

Se modifica la estructura del dataset para poder analizar el porcentaje de registros con datos por país, por feature y por año, para decidir qué features considerar para cada país.

In [ ]:
SocProtFeaturesByID = RelevantFeaturesSocProt.drop(["Country Code", "Indicator", "features"], axis=1)
SocProtFeaturesByID = SocProtFeaturesByID.groupby(["Country Name", "Indicator Code"]).count()
SocProtFeaturesByID["Porcentaje de Registros"] = (SocProtFeaturesByID.sum(axis=1)/50)*100
SocProtFeaturesByID = SocProtFeaturesByID["Porcentaje de Registros"].reset_index().sort_values(by='Indicator Code', ascending=False)

countries = social_protection['Country Name'].drop_duplicates().to_list()
for country in countries:
    plt.figure(figsize=(20, 10))
    sns.barplot(x="Indicator Code", y="Porcentaje de Registros", data=SocProtFeaturesByID[SocProtFeaturesByID["Country Name"]== country], color = "r")
    plt.xticks(fontsize=14, rotation=90)
    plt.yticks(fontsize=14)
    plt.xlabel('Indicator Code', fontsize=14)
    plt.ylabel('Porcentaje con Datos', fontsize=14)
    plt.title("Porcentaje con Datos por Indicador de " + str(country) + "\n", fontsize=16)
    plt.axhline(y=50,linewidth=1, color='black')

In [ ]:
#Se seleccionan del dataset solo aquellas columnas relevantes para el gráfico
SocProtFeaturesByID = RelevantFeaturesSocProt.drop(["Country Code", "Indicator", "features"], axis=1)
#Contamos todas las combinaciones existentes por país e indicator code para cada año
SocProtFeaturesByID = SocProtFeaturesByID.groupby(["Country Name", "Indicator Code"]).count()
#Sumamos las combinaciones de país e indicator code para todos los años, 
#Para luego, dado que la cantidad de años de análisis es 50, 
#obtener el porcentaje de registros por país e indicator code para todo el período en análisis
SocProtFeaturesByID["Porcentaje de Registros"] = (SocProtFeaturesByID.sum(axis=1)/50)*100
#Para Facilitar la lectura del cuadro, se crea una columna que identifica si el % de registros es mayor o menor a 50%
SocProtFeaturesByID["Mayor a 50%"] = SocProtFeaturesByID["Porcentaje de Registros"]>50
#Ordenamos por Indicator Code, para seleccionar las mismas variables para todos los países, y, luego, en caso que algún
# país no tenga más del 50% para algún feature, repetimos el proceso ordenando en base al % de registros.
SocProtFeaturesByID = SocProtFeaturesByID[["Porcentaje de Registros" , "Mayor a 50%"]].reset_index().sort_values(by='Indicator Code', ascending=False)
countries = gender['Country Name'].drop_duplicates()
SocProtIDbyCountry = []
for country in countries:
       SocProtIDbyCountry.append( SocProtFeaturesByID.loc[SocProtFeaturesByID["Country Name"]==country].iloc[0:15] )
SocProtIDbyCountry

In [ ]:
#Se definen los features seleccionados

SocProtIDFeatures = SocProtIDbyCountry[0]["Indicator Code"].unique()
SocProtIDFeatures

In [ ]:
                                                                                                 
print("Asignación de ID Code elegidos con su respectivo feature." )
social_protection[social_protection["Indicator Code"].isin(SocProtIDFeatures)][["Indicator Code", "features"]].drop_duplicates().reset_index().drop(columns=["index"])

#### Funciones de Densidad por Feature para cada país

Para realizar los siguientes análisis, se construye un dataset que tenga solo las columnas relevantes de análisis y que solo considere los ID Features seleccionados en el paso anterior. 

In [ ]:
columns=social_protection.columns
SocProtData=social_protection.drop(["Indicator", "Country Code", "features"], axis=1).rename(columns = lambda x : str(x)[:4] if x in columns[5:] else x)
SocProtData=SocProtData[SocProtData["Indicator Code"].isin(SocProtIDFeatures)]

In [ ]:
SocProtDataMelt = pd.melt(SocProtData, id_vars = ["Indicator Code" , "Country Name"])
SocProtDataMelt["variable"] = pd.to_numeric(SocProtDataMelt["variable"] )
SocProtDataMelt = SocProtDataMelt.rename(columns = {"variable": "Year"})

Se crea un dataset por país

In [ ]:
SocProtDataMelt = SocProtDataMelt.copy()
arg_sp=SocProtDataMelt[SocProtDataMelt["Country Name"]=="Argentina"]
bra_sp=SocProtDataMelt[SocProtDataMelt["Country Name"]=="Brasil"]
ur_sp=SocProtDataMelt[SocProtDataMelt["Country Name"]=="Uruguay"]
par_sp=SocProtDataMelt[SocProtDataMelt["Country Name"]=="Paraguay"]
ven_sp=SocProtDataMelt[SocProtDataMelt["Country Name"]=="Venezuela"]
bol_sp=SocProtDataMelt[SocProtDataMelt["Country Name"]=="Bolivia"]
chi_sp=SocProtDataMelt[SocProtDataMelt["Country Name"]=="Chile"]
guy_sp=SocProtDataMelt[SocProtDataMelt["Country Name"]=="Guyana"]
sur_sp=SocProtDataMelt[SocProtDataMelt["Country Name"]=="Suriname"]
col_sp=SocProtDataMelt[SocProtDataMelt["Country Name"]=="Colombia"]
ecu_sp=SocProtDataMelt[SocProtDataMelt["Country Name"]=="Ecuador"]
per_sp=SocProtDataMelt[SocProtDataMelt["Country Name"]=="Perú"]

Se grafican los histogramas para cada ID Features

In [ ]:
SocProtIDbyCountry = SocProtIDbyCountry.copy()
SocProtIDFeatures = SocProtIDbyCountry[0]["Indicator Code"].unique()

for Features in SocProtIDFeatures:
    plt.figure(figsize=(13,6))
    sns.distplot(arg_sp[arg_sp["Indicator Code"]==Features]["value"], label="arg", hist=False)
    sns.distplot(bra_sp[bra_sp["Indicator Code"]==Features]["value"], label="bra", hist=False)
    sns.distplot(ur_sp[ur_sp["Indicator Code"]==Features]["value"], label="ur", hist=False)
    sns.distplot(par_sp[par_sp["Indicator Code"]==Features]["value"], label="par", hist=False)
    sns.distplot(ven_sp[ven_sp["Indicator Code"]==Features]["value"], label="ven", hist=False)
    sns.distplot(bol_sp[bol_sp["Indicator Code"]==Features]["value"], label="bol", hist=False)
    sns.distplot(chi_sp[chi_sp["Indicator Code"]==Features]["value"], label="chi", hist=False)
    sns.distplot(guy_sp[guy_sp["Indicator Code"]==Features]["value"], label="guy", hist=False)
    sns.distplot(sur_sp[sur_sp["Indicator Code"]==Features]["value"], label="sur", hist=False)
    sns.distplot(col_sp[col_sp["Indicator Code"]==Features]["value"], label="col", hist=False)
    sns.distplot(ecu_sp[ecu_sp["Indicator Code"]==Features]["value"], label="ecu", hist=False)
    sns.distplot(per_sp[per_sp["Indicator Code"]==Features]["value"], label="per", hist=False)
    plt.title("Funciones de densidad "+ str(Features) + " por país", fontsize=20)
    plt.legend(loc='upper center', bbox_to_anchor=(1.4, 1), shadow=True, ncol=2, fontsize= 'xx-small')
    plt.xticks(fontsize=16, rotation=90)
    plt.yticks(fontsize=16)
    plt.xlabel('Valor', fontsize=16)
    

Como puede evidenciarse en todos los histogramas, las distribuciones de cada feature en los diferentes países difiere bastante. Es decir, en la región cada uno de los features se distribuye diferente dependiendo del país que se analice. Es posible que con un análisis mas detallado de cada feature, se encuentren países donde la feature se distribuye similar.

#### BoxPlot por Feature

Para profundizar el análisis llevado a cabo hasta ahora, se construyen Box Plot para cada feature por país. 

In [ ]:
SocProtIDbyCountry = SocProtIDbyCountry.copy()
SocProtIDFeatures = SocProtIDbyCountry[0]["Indicator Code"].unique()
SocProtDataMelt = SocProtDataMelt.copy()

for Features in SocProtIDFeatures:
    plt.figure(figsize=(13,6))
    sns.boxplot(data=SocProtDataMelt[SocProtDataMelt["Indicator Code"]==Features], x="Country Name", y="value")
    plt.xticks(fontsize=16, rotation=90)
    plt.yticks(fontsize=16)
    plt.xlabel('País', fontsize=16)
    plt.ylabel('Valor', fontsize=16)
    plt.title("Boxplot del feature "+ str(Features) + " por país", fontsize=24)

Los boxplot evidencian la presencia de outliers para muchos de los países en la mayoria de los features.

#### Series Temporales de los Features por País

In [ ]:
countries = social_protection['Country Name'].drop_duplicates().to_list()
SocProtDataMelt = SocProtDataMelt.copy()
for country in countries:
    plt.figure(figsize=(10,7))
    sns.lineplot(data=SocProtDataMelt[SocProtDataMelt["Country Name"]==country], x="Year", y="value", hue="Indicator Code")
    plt.title("Serie Temporal de los Indicadores en " + str(country), fontsize=18)
    sns.despine()
    plt.xticks(fontsize=14, rotation=90)
    plt.yticks(fontsize=14)
    plt.xlabel('País', fontsize=14)
    plt.ylabel('Valor', fontsize=14)
    plt.xticks(rotation=90)
    plt.legend(loc='upper center', bbox_to_anchor=(2, 1), shadow=True, ncol=2, fontsize= 'xx-small')

### Determinación de Año a Predecir

Una vez seleccionados los indicadores a predecir por país para la dimensión de educación, se verifica que todos los indicadores seleccionados tengan información para el año 2019, para elegirlo como año prueba. Si no se cumple la condición, entonces, se replica el análisis para el año 2018, y se opta por considerarlo como año a predecir.

In [ ]:
featureWOData_2019  = SocProtData["2019"].isna().sum()
SPData_2019 = (featureWOData_2019/len(SocProtData["2019"]))*100

print("De las 15 features seleccionadas para Protección Social, el " + str(round(SPData_2019, 2)) + "% no posee información para el año 2019." )

featureWOData_2018  = SocProtData["2018"].isna().sum()

SPData_2018 = (featureWOData_2018/len(SocProtData["2018"]))*100

print("De las 15 features seleccionadas para Protección Social, el " + str(round(SPData_2018, 2)) + "% no posee información para el año 2018." )



Teniendo el cuenta que para el año 2019 falta información en mas del 50% se decide seleccionar el año 2018

## POBREZA

1. ¿Cuántos **tipos de Features diferentes** existen por indicador considerado? ¿Todos los países mantienen esa misma cantidad para el mismo indicador?

In [ ]:
print("El indicador de Pobreza tiene " + str(len(poverty["features"].unique())) + " features.")
print("El análisis se lleva a cabo consdierando " + str(len(poverty["Country Name"].unique())) + " países.")
print(poverty["Country Name"].unique())

In [ ]:
countries = poverty['Country Name'].unique()

for country in countries:
    
    print("La cantidad de features que posee " + str(country) + ": " + str(poverty[poverty['Country Name']==country]['features'].count()) )

### Análisis por Feature para toda la Región

2.  ¿Cuál es la **proporción de valores nulos** para cada Feature dentro de cada Indicador con respecto al intervalo temporal considerado? ¿Estos valores nulos se encuentran dispersos a lo largo de todo el intervalo temporal o existen agrupamientos (más chicos y/o más grandes) de los mismos? ¿Qué decisión se podría tomar con respecto a los mismos en un futuro?
3. ¿Cómo es la distribución de valores nulos por año (por columnas del dataset)?. Teniendo en cuenta que el objetivo final de la mentoría es realizar una predicción de cada Feature para el año 2019, ¿se disponen de datos suficientes para comparar las predicciones que se obtendrán con el valor real en ese año?

In [ ]:
print("Los features de Pobreza que existen son los siguientes: \n", str(poverty['features'].unique()))

Para llevar a cabo el análisis de los datos nulos, se construye una matriz por feature y por año en la que se muestra el porcentaje de registros existentes para cada combinación considerando toda la región.

In [ ]:
PovertyValuesByICode = poverty.groupby("Indicator Code").count().drop(["Country Code", "Country Name", "Indicator", "features"], axis=1)
PovertyValuesByICode["Valor"] = PovertyValuesByICode.sum(axis=1)
PovertyValuesByICode = PovertyValuesByICode.sort_values(by=['Valor'], ascending=False)
PovertyValuesByICode = ((PovertyValuesByICode/12)*100).drop(columns = ["Valor"])
PovertyValuesByICode = PovertyValuesByICode.rename(columns = lambda x : str(x)[:4])
round(PovertyValuesByICode,2)

Se construye un mapa de calor para visualizar aquellas features con mayor porcentaje de registros, mientras mas oscuro el color menos proporción de registros hay para dicha combinación.

In [ ]:
plt.figure(figsize=(60, 30))
sns.heatmap(PovertyValuesByICode, cbar=True, linewidths=0.1,  vmin=0, vmax=100)
sns.set(font_scale=4)

Como puede observarse graficamente, durante el periodo de 1970 a 1992 se concentra la mayor cantidad de nulos.

In [ ]:
PovertyByYear = pd.melt(poverty.drop(["Country Code", "Country Name", "Indicator", "features"], axis=1), id_vars = ["Indicator Code"])
PovertyByYear["variable"] = pd.to_numeric(PovertyByYear["variable"] )
PovertyByYear = PovertyByYear.rename(columns = {"variable": "Year"})
TotalData_1992 = (PovertyByYear["Year"]<=1992).sum()
WithData_1992 = ((PovertyByYear["Year"]<=1992) & (PovertyByYear["value"]>0)).sum()

print( "El % de registros sin datos para el período de 1970 a 1992 es de: \n" +  str(round((1 - (WithData_1990/TotalData_1990))*100,2)) + "%") 

TotalData_2019 = (PovertyByYear["Year"]>1992).sum()
WithData_2019 = ((PovertyByYear["Year"]>1992) & (PovertyByYear["value"]>0)).sum()
Check = len (PovertyByYear) == (TotalData_2019 + TotalData_1990 )


print( "El % de registros sin datos para el período de 1993 a 2019 es de: \n" +  str(round((1 - (WithData_2019/TotalData_2019))*100,2)) + "%") 


Dado que la dimensión pobreza tiene 15 features en total, se analizará en detalle cada uno de ellos.

In [ ]:
PovertyByYear = PovertyByYear.copy()
PovertyByYear = PovertyByYear.reset_index()
PovertyByYear["features"] = PovertyByYear["Indicator Code"]
PovertyByYear["features"] = PovertyByYear["features"].map(poverty.set_index("Indicator Code")["features"].to_dict())
print( "La descripción de cada ID Indicator a considerar es la siguiente: ")
FeaturesPovertySelection = PovertyByYear[["Indicator Code", "features"]].drop_duplicates().set_index("Indicator Code")
FeaturesPovertySelection

### Análisis por País para cada Feature

4. Volviendo a los resultados del punto 2, elegir como mínimo 15 Features por Indicador con al menos el 50% de valores no nulos (y en lo posible los mismos para todos los países) y mostrar gráficamente, con gráficos de distribución o de cajas por ejemplo, la distribución de los mismos. ¿Qué se puede observar de estas distribuciones? ¿Son normales? ¿Qué sucede con los outliers? ¿Observan alguna similitud entre los features elegidos?. Para estos mismos 15 Features realizar el gráfico de línea de su serie temporal y analizarlo.

In [ ]:
PovertyIDCode = FeaturesPovertySelection.reset_index()['Indicator Code'].to_list()

RelevantFeaturesPoverty = poverty[poverty["Indicator Code"].isin(PovertyIDCode)]
PovertyDataByCountry = RelevantFeaturesPoverty.groupby("Country Name").count().drop(["features", "Indicator", "Country Code", "Indicator Code"], axis=1)
PovertyDataByCountry["Valor"] = PovertyDataByCountry.sum(axis=1)
PovertyDataByCountry = PovertyDataByCountry.sort_values(by=['Valor'], ascending=False)
PovertyDataByCountry = ((PovertyDataByCountry/len(RelevantFeaturesPoverty["Indicator Code"].unique()))*100).drop(columns = ["Valor"])
PovertyDataByCountry = PovertyDataByCountry.rename(columns = lambda x : str(x)[:4])
round(PovertyDataByCountry,2)

En la tabla es posible observar la falta de información en todos los países para todos los features de pobreza durante el período 1970 a 1979. En 1980 solamente en Argentina se miden algunos indicadores (estaría bueno poner cuáles), sin embargo, es solo hay algo de información para ese año luego viene un período de 5 años (1981-1985) donde nuevamente no se mide ningún indicador en Argentina.

In [ ]:
PovertyDataByCountry = PovertyDataByCountry.copy()
plt.figure(figsize=(40, 20))
sns.heatmap(PovertyDataByCountry, cbar=True, linewidths=.3, vmin=0, vmax=100)
sns.set(font_scale=4)

Por su parte en el hetmatp, se evidencia la falta de información de Guyana y Suriname para todo el período en ánalisis.

Se realiza un heatmap considerando como centro el 50% de esta forma se puede visualizar cuáles son las combinaciones que tienen mas del 50% de registros con datos.

In [ ]:
PovertyDataByCountry = PovertyDataByCountry.copy()
plt.figure(figsize=(40, 20))
sns.heatmap(PovertyDataByCountry, cbar=True, linewidths=.3, center=50, cmap="PRGn", vmin=0, vmax=100)
sns.set(font_scale=4)

Como puede observarse en el heatmap anteriormente graficado, en general existe muy poca información para el análisis de pobreza. *Pocos* cuadrantes muestran tonalidades verdes, lo que implica que en *pocos* países se tienen más del 50% de los registros con datos.

In [ ]:
PovertyDataByCountry = PovertyDataByCountry.copy()
plt.figure(figsize=(40, 20))
sns.heatmap(PovertyDataByCountry, cbar=True, linewidths=.3, center=40, cmap="PRGn", vmin=0, vmax=100)
sns.set(font_scale=4)

Con el propósito de comprender mejor la falta de datos, se realizó el hetmap anterior, donde se considera como centro 45. Implicando que aquellas combinaciones que tengan más del 45% de los registros con datos mostrarán tonalidades verdes.



In [ ]:
_porc_paises_mas_50_pov = round((PovertyDataByCountry[PovertyDataByCountry>45].count()/len(poverty["Country Name"].unique()))*100, 0).astype(int)
print("El % de países que tienen más del 45% de los features con datos para cada año es: \n" + str(_porc_paises_mas_50_pov))

In [ ]:
PovertyByYear = pd.melt(poverty, id_vars = ["features", "Indicator" , "Country Name","Country Code", "Indicator Code"])
PovertyByYear["variable"] = pd.to_numeric(PovertyByYear["variable"] )
PovertyByYear = PovertyByYear.rename(columns = {"variable": "Year"})
PovertyByYear["Year"] = PovertyByYear["Year"].apply(lambda x: str(x)[:4])

PovertyByYear

Se modifica la estructura del dataset para poder analizar el porcentaje de registros con datos por país, por feature y por año, para decidir qué features considerar para cada país.

In [ ]:
PovertyFeaturesByID = poverty.drop(["Country Code", "Indicator", "features"], axis=1)
PovertyFeaturesByID = PovertyFeaturesByID.groupby(["Country Name", "Indicator Code"]).count()
PovertyFeaturesByID["Porcentaje de Registros"] = (PovertyFeaturesByID.sum(axis=1)/50)*100
PovertyFeaturesByID = PovertyFeaturesByID["Porcentaje de Registros"].reset_index().sort_values(by='Indicator Code', ascending=False)

countries = poverty['Country Name'].drop_duplicates().to_list()

for country in countries:
    plt.figure(figsize=(20, 10))
    sns.barplot(x="Indicator Code", y="Porcentaje de Registros", data=PovertyFeaturesByID[PovertyFeaturesByID["Country Name"]== country], color = "r")
    plt.xticks(fontsize=14, rotation=90)
    plt.yticks(fontsize=14)
    plt.xlabel('Indicator Code', fontsize=14)
    plt.ylabel('Porcentaje con Datos', fontsize=14)
    plt.title("Porcentaje con Datos por Indicador de " + str(country) + "\n", fontsize=16)
    plt.gca().set_ylim(0,100)
    plt.axhline(y=50,linewidth=1, color='black')


In [ ]:
#Se seleccionan del dataset solo aquellas columnas relevantes para el gráfico
PovertyFeaturesByID = poverty.drop(["Country Code", "Indicator", "features"], axis=1)
#Contamos todas las combinaciones existentes por país e indicator code para cada año
PovertyFeaturesByID = PovertyFeaturesByID.groupby(["Country Name", "Indicator Code"]).count()
#Sumamos las combinaciones de país e indicator code para todos los años, 
#Para luego, dado que la cantidad de años de análisis es 50, 
#obtener el porcentaje de registros por país e indicator code para todo el período en análisis
PovertyFeaturesByID["Porcentaje de Registros"] = (PovertyFeaturesByID.sum(axis=1)/50)*100
#Para Facilitar la lectura del cuadro, se crea una columna que identifica si el % de registros es mayor o menor a 50%
PovertyFeaturesByID["Mayor a 50%"] = PovertyFeaturesByID["Porcentaje de Registros"]>50
#Ordenamos por Indicator Code, para seleccionar las mismas variables para todos los países, y, luego, en caso que algún
# país no tenga más del 50% para algún feature, repetimos el proceso ordenando en base al % de registros.
PovertyFeaturesByID = PovertyFeaturesByID[["Porcentaje de Registros" , "Mayor a 50%"]].reset_index().sort_values(by='Indicator Code', ascending=False)
countries = poverty['Country Name'].drop_duplicates()
PovertyIDbyCountry = []
for country in countries:
       PovertyIDbyCountry.append( PovertyFeaturesByID.loc[PovertyFeaturesByID["Country Name"]==country].iloc[0:15] )
PovertyIDbyCountry

#### Funciones de Densidad por Feature para cada país

Para realizar los siguientes análisis, se construye un dataset que tenga solo las columnas relevantes de análisis y que solo considere los ID Features seleccionados en el paso anterior. 

In [ ]:
PovertyIDFeatures = poverty["Indicator Code"].unique()
columns=poverty.columns
PovertyData=poverty.drop(["Indicator", "Country Code", "features"], axis=1).rename(columns = lambda x : str(x)[:4] if x in columns[5:] else x)
PovertyData=PovertyData[PovertyData["Indicator Code"].isin(PovertyIDFeatures)]

In [ ]:

PovertyData=PovertyData.copy()
PovertyDataMelt = pd.melt(PovertyData, id_vars = ["Indicator Code" , "Country Name"])
PovertyDataMelt["variable"] = pd.to_numeric(PovertyDataMelt["variable"] )
PovertyDataMelt = PovertyDataMelt.rename(columns = {"variable": "Year"})

Se crea un dataset por país

In [ ]:
PovertyDataMelt = PovertyDataMelt.copy()

arg_pov=PovertyDataMelt[PovertyDataMelt["Country Name"]=="Argentina"]
bra_pov=PovertyDataMelt[PovertyDataMelt["Country Name"]=="Brasil"]
ur_pov=PovertyDataMelt[PovertyDataMelt["Country Name"]=="Uruguay"]
par_pov=PovertyDataMelt[PovertyDataMelt["Country Name"]=="Paraguay"]
ven_pov=PovertyDataMelt[PovertyDataMelt["Country Name"]=="Venezuela"]
bol_pov=PovertyDataMelt[PovertyDataMelt["Country Name"]=="Bolivia"]
chi_pov=PovertyDataMelt[PovertyDataMelt["Country Name"]=="Chile"]
guy_pov=PovertyDataMelt[PovertyDataMelt["Country Name"]=="Guyana"]
sur_pov=PovertyDataMelt[PovertyDataMelt["Country Name"]=="Suriname"]
col_pov=PovertyDataMelt[PovertyDataMelt["Country Name"]=="Colombia"]
ecu_pov=PovertyDataMelt[PovertyDataMelt["Country Name"]=="Ecuador"]
per_pov=PovertyDataMelt[PovertyDataMelt["Country Name"]=="Perú"]

Se grafican los histogramas para cada ID Features

In [ ]:
PovertyIDbyCountry = PovertyIDbyCountry.copy()
PovertyIDFeatures = PovertyIDbyCountry[0]["Indicator Code"].unique()

for PovertyIDFeatures in PovertyIDFeatures:
    plt.figure(figsize=(13,6))
    sns.distplot(arg_pov[arg_pov["Indicator Code"]==PovertyIDFeatures]["value"], label="arg", hist=False)
    sns.distplot(bra_pov[bra_pov["Indicator Code"]==PovertyIDFeatures]["value"], label="bra", hist=False)
    sns.distplot(ur_pov[ur_pov["Indicator Code"]==PovertyIDFeatures]["value"], label="ur", hist=False)
    sns.distplot(par_pov[par_pov["Indicator Code"]==PovertyIDFeatures]["value"], label="par", hist=False)
    sns.distplot(ven_pov[ven_pov["Indicator Code"]==PovertyIDFeatures]["value"], label="ven", hist=False)
    sns.distplot(bol_pov[bol_pov["Indicator Code"]==PovertyIDFeatures]["value"], label="bol", hist=False)
    sns.distplot(chi_pov[chi_pov["Indicator Code"]==PovertyIDFeatures]["value"], label="chi", hist=False)
    sns.distplot(guy_pov[guy_pov["Indicator Code"]==PovertyIDFeatures]["value"], label="guy", hist=False)
    sns.distplot(sur_pov[sur_pov["Indicator Code"]==PovertyIDFeatures]["value"], label="sur", hist=False)
    sns.distplot(col_pov[col_pov["Indicator Code"]==PovertyIDFeatures]["value"], label="col", hist=False)
    sns.distplot(ecu_pov[ecu_pov["Indicator Code"]==PovertyIDFeatures]["value"], label="ecu", hist=False)
    sns.distplot(per_pov[per_pov["Indicator Code"]==PovertyIDFeatures]["value"], label="per", hist=False)
    plt.title("Funciones de densidad "+ str(PovertyIDFeatures) + " por país", fontsize=20)
    plt.legend(loc='upper center', bbox_to_anchor=(1.4, 1), shadow=True, ncol=2, fontsize= 'xx-small')
    plt.xticks(fontsize=16, rotation=90)
    plt.yticks(fontsize=16)
    plt.xlabel('Valor', fontsize=16)

A pesar de la falta de información, puede observarse que las distribuciones de cada feature en los diferentes países difiere bastante.

#### BoxPlot por Feature

Para profundizar el análisis llevado a cabo hasta ahora, se construyen Box Plot para cada feature por país.

In [ ]:
PovertyIDbyCountry = PovertyIDbyCountry.copy()
PovertyIDFeatures = PovertyIDbyCountry[0]["Indicator Code"].unique()
PovertyDataMelt = PovertyDataMelt.copy()

for Features in PovertyIDFeatures:
    plt.figure(figsize=(13,6))
    sns.boxplot(data=PovertyDataMelt[PovertyDataMelt["Indicator Code"]==Features], x="Country Name", y="value")
    plt.title("Boxplot del feature "+ str(Features) + " por país", fontsize=24)
    plt.legend(loc='upper center', bbox_to_anchor=(1.4, 1), shadow=True)
    plt.xticks(fontsize=16, rotation=90)
    plt.yticks(fontsize=16)
    plt.xlabel('País', fontsize=16)
    plt.ylabel('Valor', fontsize=16)


#### Series Temporales de los Features por País

In [ ]:
countries = poverty['Country Name'].drop_duplicates().to_list()
PovertyDataMelt = PovertyDataMelt.copy()
for country in countries:
    plt.figure(figsize=(10,7))
    sns.lineplot(data=PovertyDataMelt[PovertyDataMelt["Country Name"]==country], x="Year", y="value", hue="Indicator Code")
    plt.title("Serie Temporal de los Indicadores en " + str(country), fontsize=18)
    sns.despine()
    plt.xticks(fontsize=14, rotation=90)
    plt.yticks(fontsize=14)
    plt.xlabel('País', fontsize=14)
    plt.ylabel('Valor', fontsize=14)
    plt.legend(loc='upper center', bbox_to_anchor=(2, 1), shadow=True, ncol=2, fontsize= 'xx-small')

### Determinación de Año a Predecir

Una vez seleccionados los indicadores a predecir por país para la dimensión de educación, se verifica que todos los indicadores seleccionados tengan información para el año 2019, para elegirlo como año prueba. Si no se cumple la condición, entonces, se replica el análisis para el año 2018, y se opta por considerarlo como año a predecir.

In [ ]:
featureWOData_2019  = PovertyData["2019"].isna().sum()
POData_2019 = (featureWOData_2019/len(PovertyData["2019"]))*100

print("De las 15 features para Pobreza, el \n" + str(round(POData_2019, 2)) + "% no posee información para el año 2019." )

featureWOData_2018  = PovertyData["2018"].isna().sum()

POData_2018 = (featureWOData_2018/len(PovertyData["2018"]))*100

print("De las 15 features para Pobreza, el \n" + str(round(POData_2018, 2)) + "% no posee información para el año 2018." )



Teniendo el cuenta que para el año 2019 falta información, se decide seleccionar el año 2018.

## MEDIO AMBIENTE

1. ¿Cuántos **tipos de Features diferentes** existen por indicador considerado? ¿Todos los países mantienen esa misma cantidad para el mismo indicador?

In [ ]:
print("El indicador de Medio Ambiente tiene " + str(len(environment["features"].unique())) + " features.")
print("El análisis se lleva a cabo consdierando " + str(len(environment["Country Name"].unique())) + " países.")
print(environment["Country Name"].unique())

In [ ]:
countries = environment['Country Name'].unique()

for countries in countries:
    
    print("La cantidad de features que posee " + str(countries) + ": " + str(environment[environment['Country Name']==countries]['features'].count()) )

### Análisis por Feature para toda la Región

2.  ¿Cuál es la **proporción de valores nulos** para cada Feature dentro de cada Indicador con respecto al intervalo temporal considerado? ¿Estos valores nulos se encuentran dispersos a lo largo de todo el intervalo temporal o existen agrupamientos (más chicos y/o más grandes) de los mismos? ¿Qué decisión se podría tomar con respecto a los mismos en un futuro?
3. ¿Cómo es la distribución de valores nulos por año (por columnas del dataset)?. Teniendo en cuenta que el objetivo final de la mentoría es realizar una predicción de cada Feature para el año 2019, ¿se disponen de datos suficientes para comparar las predicciones que se obtendrán con el valor real en ese año?

In [ ]:
print("Los features de Medio Ambiente que existen son los siguientes: \n", str(environment['features'].unique()))

Para llevar a cabo el análisis de los datos nulos, se construye una matriz por feature y por año en la que se muestra el porcentaje de registros existentes para cada combinación considerando toda la región.

In [ ]:
EnviroValuesByICode = environment.groupby("Indicator Code").count().drop(["Country Code", "Country Name", "Indicator", "features"], axis=1)
EnviroValuesByICode["Valor"] = EnviroValuesByICode.sum(axis=1)
EnviroValuesByICode = EnviroValuesByICode.sort_values(by=['Valor'], ascending=False)
EnviroValuesByICode = ((EnviroValuesByICode/12)*100).drop(columns = ["Valor"])
EnviroValuesByICode = EnviroValuesByICode.rename(columns = lambda x : str(x)[:4])
round(EnviroValuesByICode,2)

Se construye un mapa de calor para visualizar aquellas features con mayor porcentaje de registros, mientras mas oscuro el color menos proporción de registros hay para dicha combinación.

In [ ]:
plt.figure(figsize=(60, 30))
sns.heatmap(EnviroValuesByICode, cbar=True, linewidths=0.1,  vmin=0, vmax=100)
sns.set(font_scale=4)

Como se observa en el grafico, durante el periodo de 1970 a 1990 para ciertos indicadores se concentra la mayor cantidad de nulos. Es importante destacar que desde el año 2009 hasta la actualidad empieza a crecer la cantidad de datos nulos de los indicadores. Para el año 2013 ya hay una gran cantidad de datos nulos que sigue creciendo hasta el punto en el cual no hay indicador que tenga informacion en el año 2019, lo que condicionará nuestra predicción.

In [ ]:
EnviroByYear = pd.melt(environment.drop(["Country Code", "Country Name", "Indicator", "features"], axis=1), id_vars = ["Indicator Code"])
EnviroByYear["variable"] = pd.to_numeric(EnviroByYear["variable"] )
EnviroByYear = EnviroByYear.rename(columns = {"variable": "Year"})
TotalData_1990 = (EnviroByYear["Year"]<1990).sum()
WithData_1990 = ((EnviroByYear["Year"]<1990) & (EnviroByYear["value"]>0)).sum()

print("El % de registros sin datos para el período de 1970 a 1990 es de: \n" +  str(round((1 - (WithData_1990/TotalData_1990))*100,2)) + "%") 

TotalData_2008 = ((1990<=EnviroByYear["Year"])&(EnviroByYear["Year"]<=2008)).sum()
WithData_2008 = (((1990<=EnviroByYear["Year"])&(EnviroByYear["Year"]<=2008)) & (EnviroByYear["value"]>0)).sum()

print("El % de registros sin datos para el período de 1991 a 2008 es de: \n" +  str(round((1 - (WithData_2008/TotalData_2008))*100,2)) + "%") 

TotalData_2019 = (EnviroByYear["Year"]>=2009).sum()
WithData_2019 = ((EnviroByYear["Year"]>=2009) & (EnviroByYear["value"]>0)).sum()

Check = len (EnviroByYear) == (TotalData_2019 + TotalData_2008 + TotalData_1990 )

print("El % de registros sin datos para el período de 2009 a 2019 es de: \n" +  str(round((1 - (WithData_2019/TotalData_2019))*100,2)) + "%") 

Se seleccionan en un principio los 30 features con mas datos en toda la región:

In [ ]:
EnviroByYear = EnviroByYear.reset_index()
EnviroByYear["features"] = EnviroByYear["Indicator Code"]
EnviroByYear["features"] = EnviroByYear["features"].map(environment.set_index("Indicator Code")["features"].to_dict())
FeaturesEnviroSelection = EnviroByYear.iloc[0:15][["Indicator Code", "features"]].set_index("Indicator Code")
FeaturesEnviroSelection

### Análisis por País para cada Feature

4. Volviendo a los resultados del punto 2, elegir como mínimo 15 Features por Indicador con al menos el 50% de valores no nulos (y en lo posible los mismos para todos los países) y mostrar gráficamente, con gráficos de distribución o de cajas por ejemplo, la distribución de los mismos. ¿Qué se puede observar de estas distribuciones? ¿Son normales? ¿Qué sucede con los outliers? ¿Observan alguna similitud entre los features elegidos?. Para estos mismos 15 Features realizar el gráfico de línea de su serie temporal y analizarlo.

Se construye una matriz para determinar cuál es el porcentaje de registros con datos considerando las 30 features seleccionadas por país y año. 

In [ ]:
EnviroIDCode = FeaturesEnviroSelection.reset_index()['Indicator Code'].to_list()
RelevantFeaturesEnviro = environment[environment["Indicator Code"].isin(EnviroIDCode)]

EnviroDataByCountry = RelevantFeaturesEnviro.groupby("Country Name").count().drop(["features", "Indicator", "Country Code", "Indicator Code"], axis=1)
EnviroDataByCountry["Valor"] = EnviroDataByCountry.sum(axis=1)
EnviroDataByCountry = EnviroDataByCountry.sort_values(by=['Valor'], ascending=False)
EnviroDataByCountry = ((EnviroDataByCountry/len(RelevantFeaturesEnviro["Indicator Code"].unique()))*100).drop(columns = ["Valor"])
EnviroDataByCountry = EnviroDataByCountry.rename(columns = lambda x : str(x)[:4])
round(EnviroDataByCountry,2)

In [ ]:
plt.figure(figsize=(40, 20))
sns.heatmap(EnviroDataByCountry, cbar=True, linewidths=.3, vmin=0, vmax=100)
sns.set(font_scale=4)

Se realiza un heatmap considerando como centro el 50% de esta forma se puede visualizar cuáles son las combinaciones que tienen mas del 50% de registros con datos.

In [ ]:
plt.figure(figsize=(40, 20))
sns.heatmap(EnviroDataByCountry, cbar=True, linewidths=.3, center=50, cmap="PRGn")
sns.set(font_scale=4)

In [ ]:
_porc_paises_mas_50_env = round((EnviroDataByCountry[EnviroDataByCountry>50].count()/len(environment["Country Name"].unique()))*100, 0).astype(int)

print("El % de países que tienen más del 50% de los features con datos para cada año es: \n" + str(_porc_paises_mas_50_env))

In [ ]:
EnviroByYear = pd.melt(RelevantFeaturesEnviro, id_vars = ["features", "Indicator" , "Country Name","Country Code", "Indicator Code"])
EnviroByYear["variable"] = pd.to_numeric(EnviroByYear["variable"] )
EnviroByYear = EnviroByYear.rename(columns = {"variable": "Year"})
EnviroByYear["Year"] = EnviroByYear["Year"].apply(lambda x: str(x)[:4])

EnviroByYear

A partir de 1991, todos los paises en análisis tienen información para más del 50% de los features relevantes. Esto se evidencia en el heatmap, donde aquellas combinaciones con más del 50% de registros se muestran en diferentes tonalidades de verdes. A partir del 2015 abundan las conmbinaciones que no alcanzan el 50% de los registros, lo que se visualiza al irse perdiendo el color verde y apareciendo las diferentes tonalidades de violeta.

Se modifica la estructura del dataset para poder analizar el porcentaje de registros con datos por país, por feature y por año, para decidir qué features considerar para cada país.

In [ ]:
RelevantFeaturesByID = RelevantFeaturesEnviro.drop(["Country Code", "Indicator", "features"], axis=1)
RelevantFeaturesByID = RelevantFeaturesByID.groupby(["Country Name", "Indicator Code"]).count()
RelevantFeaturesByID["Porcentaje de Registros"] = (RelevantFeaturesByID.sum(axis=1)/50)*100
# RelevantFeaturesByID = RelevantFeaturesByID["Porcentaje de Registros"].reset_index().sort_values(by='Porcentaje de Registros', ascending=True)
RelevantFeaturesByID = RelevantFeaturesByID["Porcentaje de Registros"].reset_index().sort_values(by='Indicator Code', ascending=False)

countries = environment['Country Name'].drop_duplicates().to_list()
for countries in countries:
    plt.figure(figsize=(20, 10))
    sns.barplot(x="Indicator Code", y="Porcentaje de Registros", data=RelevantFeaturesByID[RelevantFeaturesByID["Country Name"]== countries], color = "r")
    plt.xticks(fontsize=14, rotation=90)
    plt.yticks(fontsize=14)
    plt.xlabel('Indicator Code', fontsize=14)
    plt.ylabel('Porcentaje con Datos', fontsize=14)
    plt.title("Porcentaje con Datos por Indicador de " + str(countries) + "\n", fontsize=16)
    plt.axhline(y=50,linewidth=1, color='black')


In [ ]:
#Se seleccionan del dataset solo aquellas columnas relevantes para el gráfico
EnviroRelevantFeaturesByID = RelevantFeaturesEnviro.drop(["Country Code", "Indicator", "features"], axis=1)
#Contamos todas las combinaciones existentes por país e indicator code para cada año
EnviroRelevantFeaturesByID = EnviroRelevantFeaturesByID.groupby(["Country Name", "Indicator Code"]).count()
#Sumamos las combinaciones de país e indicator code para todos los años, 
#Para luego, dado que la cantidad de años de análisis es 50, 
#obtener el porcentaje de registros por país e indicator code para todo el período en análisis
EnviroRelevantFeaturesByID["Porcentaje de Registros"] = (EnviroRelevantFeaturesByID.sum(axis=1)/50)*100
#Para Facilitar la lectura del cuadro, se crea una columna que identifica si el % de registros es mayor o menor a 50%
EnviroRelevantFeaturesByID["Mayor a 50%"] = EnviroRelevantFeaturesByID["Porcentaje de Registros"]>50
#Ordenamos por Indicator Code, para seleccionar las mismas variables para todos los países, y, luego, en caso que algún
# país no tenga más del 50% para algún feature, repetimos el proceso ordenando en base al % de registros.
EnviroRelevantFeaturesByID = EnviroRelevantFeaturesByID[["Porcentaje de Registros" , "Mayor a 50%"]].reset_index().sort_values(by='Indicator Code', ascending=False)
countries = environment['Country Name'].drop_duplicates()
EnviroIDbyCountry = []
for countries in countries:
       EnviroIDbyCountry.append( EnviroRelevantFeaturesByID.loc[EnviroRelevantFeaturesByID["Country Name"]==countries].iloc[0:15] )
EnviroIDbyCountry

De la salida anterior se desprende que no todos los features seleccionados posee más del 50% de datos para todos los países. En particular, para Paraguay, Venezuela, Guyana y Suriname tendremos que considerar features adicionales. Se prosigue armando una lista general que contenga la lista de features anterior con algunos adicionales en funcion de los features con más datos para estos ultimos países.

In [ ]:
# Buscamos primeros 15 features con mas datos para Paraguay

ParEnviroValuesByICode = environment[environment["Country Name"]=="Paraguay"].groupby("Indicator Code").count().drop(["Country Code", "Indicator", "features"], axis=1)
ParEnviroValuesByICode["Valor"] = ParEnviroValuesByICode.sum(axis=1)
ParEnviroValuesByICode = ParEnviroValuesByICode.sort_values(by=['Valor'], ascending=False)
ParEnviroValuesByICode = ((ParEnviroValuesByICode/12)*100).drop(columns = ["Valor"])
ParEnviroValuesByICode = ParEnviroValuesByICode.rename(columns = lambda x : str(x)[:4])

ParEnviroValuesByICode.reset_index().iloc[0:15]["Indicator Code"].to_list()

In [ ]:
# Buscamos primeros 15 features con mas datos para Venezuela

VenEnviroValuesByICode = environment[environment["Country Name"]=="Venezuela"].groupby("Indicator Code").count().drop(["Country Code", "Indicator", "features"], axis=1)
VenEnviroValuesByICode["Valor"] = VenEnviroValuesByICode.sum(axis=1)
VenEnviroValuesByICode = VenEnviroValuesByICode.sort_values(by=['Valor'], ascending=False)
VenEnviroValuesByICode = ((VenEnviroValuesByICode/12)*100).drop(columns = ["Valor"])
VenEnviroValuesByICode = VenEnviroValuesByICode.rename(columns = lambda x : str(x)[:4])

VenEnviroValuesByICode.reset_index().iloc[0:15]["Indicator Code"].to_list()

In [ ]:
# Buscamos primeros 15 features con mas datos para Guyana

GuyEnviroValuesByICode = environment[environment["Country Name"]=="Guyana"].groupby("Indicator Code").count().drop(["Country Code", "Indicator", "features"], axis=1)
GuyEnviroValuesByICode["Valor"] = GuyEnviroValuesByICode.sum(axis=1)
GuyEnviroValuesByICode = GuyEnviroValuesByICode.sort_values(by=['Valor'], ascending=False)
GuyEnviroValuesByICode = ((GuyEnviroValuesByICode/12)*100).drop(columns = ["Valor"])
GuyEnviroValuesByICode = GuyEnviroValuesByICode.rename(columns = lambda x : str(x)[:4])

GuyEnviroValuesByICode.reset_index().iloc[0:15]["Indicator Code"].to_list()

In [ ]:
# Buscamos primeros 15 features con mas datos para Suriname

SurEnviroValuesByICode = environment[environment["Country Name"]=="Suriname"].groupby("Indicator Code").count().drop(["Country Code", "Indicator", "features"], axis=1)
SurEnviroValuesByICode["Valor"] = SurEnviroValuesByICode.sum(axis=1)
SurEnviroValuesByICode = SurEnviroValuesByICode.sort_values(by=['Valor'], ascending=False)
SurEnviroValuesByICode = ((SurEnviroValuesByICode/12)*100).drop(columns = ["Valor"])
SurEnviroValuesByICode = SurEnviroValuesByICode.rename(columns = lambda x : str(x)[:4])

SurEnviroValuesByICode.reset_index().iloc[0:15]["Indicator Code"].to_list()

Como Paraguay tenía 8 False, deberiamos agregar 8 features por él. Visualizando sus primeros 15 features con más datos, que no coincidan con la lista original, elegimos los siguientes: 
"AG.LND.TOTL.K2", 
"AG.SRF.TOTL.K2", 
"NY.ADJ.DKAP.CD'", 
"NY.ADJ.AEDU.GN.ZS", 
"NY.ADJ.DCO2.CD", 
"NY.ADJ.DFOR.CD", 
"NY.ADJ.DMIN.CD", 
"ER.FSH.PROD.MT".

Venezuela debería agregar 4 features, y dentro de su lista se encuentran features coincidentes con los de Paraguay, de manera que no agregamos ningun extra por el. En el caso de Guyana deberiamos agregar dos y sucede lo mismo que antes, de modo que tampoco agregamos ninguno por el. 

En el caso de Suriname, por su parte, deberiamos agregar 8 features. Visualizando sus primeros 15 features con más datos se observa que 7 de ellos coinciden con los de Paraguay. Consecuentemente, solo debemos agregar 1 y se escoje:
"NY.ADJ.AEDU.CD"

In [ ]:
#Se definen los features seleccionados

EnviroIDbyCountry = EnviroIDbyCountry.copy()
EnviroIDFeatures = EnviroIDbyCountry[0]["Indicator Code"].to_list() +  [ "AG.LND.TOTL.K2", "AG.SRF.TOTL.K2", "NY.ADJ.DKAP.CD'", "NY.ADJ.AEDU.GN.ZS", "NY.ADJ.DCO2.CD", "NY.ADJ.DFOR.CD", "NY.ADJ.DMIN.CD", "ER.FSH.PROD.MT", "NY.ADJ.AEDU.CD"]
EnviroIDFeatures

In [ ]:
print("Asignación de ID Code elegidos con su respectivo feature." )
environment[environment["Indicator Code"].isin(EnviroIDFeatures)][["Indicator Code", "features"]].drop_duplicates().reset_index().drop(columns=["index"])

#### Funciones de Densidad por Feature para cada país

Para realizar los siguientes análisis, se construye un dataset que tenga solo las columnas relevantes de análisis y que solo considere los ID Features seleccionados en el paso anterior. 

In [ ]:
columns=environment.columns
EnviroData=environment.drop(["Indicator", "Country Code", "features"], axis=1).rename(columns = lambda x : str(x)[:4] if x in columns[5:] else x)
EnviroData=EnviroData[EnviroData["Indicator Code"].isin(EnviroIDFeatures)]

In [ ]:
EnviroDataMelt = pd.melt(EnviroData, id_vars = ["Indicator Code" , "Country Name"])
EnviroDataMelt["variable"] = pd.to_numeric(EnviroDataMelt["variable"] )
EnviroDataMelt = EnviroDataMelt.rename(columns = {"variable": "Year"})

Se crea un dataset por país

In [ ]:
arg_env=EnviroDataMelt[EnviroDataMelt["Country Name"]=="Argentina"]
bra_env=EnviroDataMelt[EnviroDataMelt["Country Name"]=="Brasil"]
ur_env=EnviroDataMelt[EnviroDataMelt["Country Name"]=="Uruguay"]
par_env=EnviroDataMelt[EnviroDataMelt["Country Name"]=="Paraguay"]
ven_env=EnviroDataMelt[EnviroDataMelt["Country Name"]=="Venezuela"]
bol_env=EnviroDataMelt[EnviroDataMelt["Country Name"]=="Bolivia"]
chi_env=EnviroDataMelt[EnviroDataMelt["Country Name"]=="Chile"]
guy_env=EnviroDataMelt[EnviroDataMelt["Country Name"]=="Guyana"]
sur_env=EnviroDataMelt[EnviroDataMelt["Country Name"]=="Suriname"]
col_env=EnviroDataMelt[EnviroDataMelt["Country Name"]=="Colombia"]
ecu_env=EnviroDataMelt[EnviroDataMelt["Country Name"]=="Ecuador"]
per_env=EnviroDataMelt[EnviroDataMelt["Country Name"]=="Perú"]

Se grafican los histogramas para cada ID Features

In [ ]:
for Features in EnviroIDFeatures:
    plt.figure(figsize=(13,6))
    sns.distplot(arg_env[arg_env["Indicator Code"]==Features]["value"], label="arg", hist=False)
    sns.distplot(bra_env[bra_env["Indicator Code"]==Features]["value"], label="bra", hist=False)
    sns.distplot(ur_env[ur_env["Indicator Code"]==Features]["value"], label="ur", hist=False)
    sns.distplot(par_env[par_env["Indicator Code"]==Features]["value"], label="par", hist=False)
    sns.distplot(ven_env[ven_env["Indicator Code"]==Features]["value"], label="ven", hist=False)
    sns.distplot(bol_env[bol_env["Indicator Code"]==Features]["value"], label="bol", hist=False)
    sns.distplot(chi_env[chi_env["Indicator Code"]==Features]["value"], label="chi", hist=False)
    sns.distplot(guy_env[guy_env["Indicator Code"]==Features]["value"], label="guy", hist=False)
    sns.distplot(sur_env[sur_env["Indicator Code"]==Features]["value"], label="sur", hist=False)
    sns.distplot(col_env[col_env["Indicator Code"]==Features]["value"], label="col", hist=False)
    sns.distplot(ecu_env[ecu_env["Indicator Code"]==Features]["value"], label="ecu", hist=False)
    sns.distplot(per_env[per_env["Indicator Code"]==Features]["value"], label="per", hist=False)
    plt.title("Funciones de densidad "+ str(Features) + " por país", fontsize=20)
    plt.legend(loc='upper center', bbox_to_anchor=(1.4, 1), shadow=True, ncol=2, fontsize= 'xx-small')
    plt.xticks(fontsize=16, rotation=90)
    plt.yticks(fontsize=16)
    plt.xlabel('Valor', fontsize=16)
    

En el caso de Environment, no se observa una heterogeneidad tan grande entre las distribuciones de las features entre países como para otros indicadores. Sin embargo, existen algunas diferencias para casos particulares de features. El país de la región que pareciera tener un comportamiento sustanciamente distinto al de sus pares de la región para muchos features es Suriname.

#### BoxPlot por Feature

Para profundizar el análisis llevado a cabo hasta ahora, se construyen Box Plot para cada feature por país. 

In [ ]:
EnviroIDbyCountry = EnviroIDbyCountry.copy()
EnviroIDFeatures = EnviroIDbyCountry[0]["Indicator Code"].unique()
EnviroDataMelt = EnviroDataMelt.copy()

for Features in EnviroIDFeatures:
    plt.figure(figsize=(13,6))
    sns.boxplot(data=EnviroDataMelt[EnviroDataMelt["Indicator Code"]==Features], x="Country Name", y="value")
    plt.title("Boxplot del feature "+ str(Features) + " por país", fontsize=24)
    plt.xticks(fontsize=16, rotation=90)
    plt.yticks(fontsize=16)
    plt.xlabel('País', fontsize=16)
    plt.ylabel('Valor', fontsize=16)

El analisis del boxplot permite observar las grandes diferencias que existen entre países para muchos de los features en cuanto a sus medidas de posición. Algunos países presentan un gran rango de datos y otros países tienen mayor concentración de los mismos, dependiendo del feature particular. Brasil, Bolivia y Venezuela en general son países de mayor dispersión de datos, respecto a otros como Argentina o Uruguay que para ningún feature presentan gran variabilidad. 

#### Series Temporales de los Features por País

In [ ]:
countries = environment['Country Name'].drop_duplicates().to_list()

for country in countries:
    plt.figure(figsize=(10,7))
    sns.lineplot(data=EnviroDataMelt[EnviroDataMelt["Country Name"]==country], x="Year", y="value", hue="Indicator Code")
    plt.title("Serie Temporal de los Indicadores en " + str(country), fontsize=18)
    sns.despine()
    plt.xticks(fontsize=14, rotation=90)
    plt.yticks(fontsize=14)
    plt.xlabel('País', fontsize=14)
    plt.ylabel('Valor', fontsize=14)
    plt.legend(loc='upper center', bbox_to_anchor=(2, 1), shadow=True, ncol=2, fontsize= 'xx-small')

### Determinación del Año a Predecir

Una vez seleccionados los indicadores a predecir por país para la dimensión de educación, se verifica que todos los indicadores seleccionados tengan información para el año 2019, para elegirlo como año prueba. Si no se cumple la condición, entonces, se replica el análisis para años anteriores hasta encontrar un año con la totalidad de datos.

In [ ]:
featureWOData_2019_Env  = EnviroData["2019"].isna().sum()
Data_2019_Env = (featureWOData_2019_Env/len(EnviroData["2019"]))*100
print("De las features seleccionadas para medio ambiente, el " + str(round(Data_2019_Env, 2)) + "% no posee información para el año 2019." )

featureWOData_2018_Env  = EnviroData["2018"].isna().sum()
Data_2018_Env = (featureWOData_2018_Env/len(EnviroData["2018"]))*100
print("De las features seleccionadas para medio ambiente, el " + str(Data_2018_Env) + "% no posee información para el año 2018." )

featureWOData_2017_Env  = EnviroData["2017"].isna().sum()
Data_2017_Env = (featureWOData_2017_Env/len(EnviroData["2017"]))*100
print("De las features seleccionadas para medio ambiente, el " + str(Data_2017_Env) + "% no posee información para el año 2017." )


En este caso convendría estimar el año 2018.

## ANÁLISIS GENERAL

### Selección de 5 features preferidos

Elegir un Indicador de su interés y 5 features relacionados al mismo, también de su interés, y trazar la serie temporal sus valores a lo largo de los años. ¿Se observa alguna tendencia o algún patrón que se repite cada tanto?



In [ ]:
# Seleccion de las features de interés
preferred_features_list=["Tasa de mortalidad, adultos, mujeres (por cada 1.000 mujeres adultas)", "Tasa de mortalidad, adultos, varones (por cada 1.000 varones adultos)",
                         "Tasa de incidencia de la pobreza, sobre la base de la línea de pobreza nacional (% de la población)",
                         'Desempleo, varones (% de la población activa masculina) (estimación modelado OIT)',
                         'Desempleo, mujeres (% de la población activa femenina) (estimación modelado OIT)']
preferred_features_list

In [ ]:
#Se construye dataset para graficar heatmap
AllValuesByICode = dataset[dataset["features"].isin(preferred_features_list)]
AllValuesByICode = AllValuesByICode.drop(["Indicator", "Indicator Code", "Country Code", "Country Name"], axis=1).drop_duplicates()
AllValuesByICode = AllValuesByICode.groupby("features").count()
AllValuesByICode["Valor"] = AllValuesByICode.sum(axis=1)
AllValuesByICode = ((AllValuesByICode/12)*100).drop(columns = ["Valor"])
AllValuesByICode = AllValuesByICode.rename(columns = lambda x : str(x)[:4])
AllValuesByICode = round(AllValuesByICode,2)

# Se cambia acorta el nombre de de los features para una mejor visualización
AllValuesByICode = AllValuesByICode.rename(index={ "Tasa de mortalidad, adultos, mujeres (por cada 1.000 mujeres adultas)": "Tasa de mortalidad, Mujeres" ,
                         "Tasa de mortalidad, adultos, varones (por cada 1.000 varones adultos)" : "Tasa de mortalidad, Varones" ,
                         "Tasa de incidencia de la pobreza, sobre la base de la línea de pobreza nacional (% de la población)" : "Tasa de incidencia de la pobreza" ,
                         "Desempleo, varones (% de la población activa masculina) (estimación modelado OIT)" : "Desempleo, Varones",
                         "Desempleo, mujeres (% de la población activa femenina) (estimación modelado OIT)": "Desempleo, Mujeres"}, inplace=False)

In [ ]:
plt.figure(figsize=(40, 20))
sns.heatmap(AllValuesByICode, cbar=True, linewidths=.3, center=50, cmap="PRGn")
sns.set(font_scale=4)

De los 5 features seleccionados, puede observarse que:
* La **tasa de mortalidad por género**, tiene información para todos los países en todo el periódo de análisis.

* La **tasa de desempleo por género**, empieza a medirse recién a partir de 1990.

* La **tasa de incidencia de la pobreza**, empieza medirse en algunos países a partir de 1997. Durante el período de 1970-1997 no existe información de la incidencia de la pobreza en los países del Mercosur.

Esto implica que aquellos indicadores seleccionados relacionados a cuestiones socio-económicas empezaron a medirse oficialmente mas tarde que los indicadores demográficos  y de salud.




In [ ]:
# Se modifica la estructura del dataset para poder graficarlo

preferred_features_dataset = dataset.drop(["Indicator", "Country Code", "Indicator Code"], axis=1).rename(columns = lambda x : str(x)[:4] if x in columns[5:] else x)
preferred_features_dataset_alt = pd.melt(preferred_features_dataset[preferred_features_dataset["features"].isin(preferred_features_list)], id_vars = ["features" , "Country Name"])
preferred_features_dataset_alt["variable"] = pd.to_numeric(preferred_features_dataset_alt["variable"] )
preferred_features_dataset_alt = preferred_features_dataset_alt.rename(columns = {"variable": "Year"})

# Se eliminan las combinaciones repetidas
preferred_features_dataset_alt = preferred_features_dataset_alt.drop_duplicates()

# Se cambia acorta el nombre de de los features para una mejor visualización
preferred_features_dataset_alt["features"] = preferred_features_dataset_alt["features"].replace({ "Tasa de mortalidad, adultos, mujeres (por cada 1.000 mujeres adultas)": "Tasa de mortalidad, Mujeres" ,
                         "Tasa de mortalidad, adultos, varones (por cada 1.000 varones adultos)" : "Tasa de mortalidad, Varones" ,
                         "Tasa de incidencia de la pobreza, sobre la base de la línea de pobreza nacional (% de la población)" : "Tasa de incidencia de la pobreza" ,
                         "Desempleo, varones (% de la población activa masculina) (estimación modelado OIT)" : "Desempleo, Varones",
                         "Desempleo, mujeres (% de la población activa femenina) (estimación modelado OIT)": "Desempleo, Mujeres"}, inplace=False)

In [ ]:
mortality_features = ["Tasa de mortalidad, Mujeres" , "Tasa de mortalidad, Varones"] 

for Features in mortality_features:
    plt.figure(figsize=(20,10))
    sns.lineplot(data=preferred_features_dataset_alt[preferred_features_dataset_alt["features"]==Features], x="Year", y="value", hue="Country Name")
    plt.title("Series Temporales de los Features en \n" + str(Features))
    plt.xticks(rotation=90)
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., fontsize= 'xx-small')

En las líneas de tiempo graficadas anteriormente, se evidencia de la tandencia a la baja de la tasa de mortalidad tanto de hombres como mujeres para todos los países del Mercosur.

In [ ]:
countries = dataset['Country Name'].drop_duplicates().to_list()
mortality_dataset = preferred_features_dataset_alt[preferred_features_dataset_alt["features"].isin(mortality_features)]

for country in countries:
    plt.figure(figsize=(20,10))
    sns.lineplot(data=mortality_dataset[mortality_dataset["Country Name"]==country], x="Year", y="value", hue="features")
    plt.title("Series Temporales de las tasas de mortalidad en \n" + str(country))
    plt.xticks(rotation=90)
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., fontsize= 'xx-small')

A su vez, en los doce países en análisis, la tasa de mortalidad masculina es mayor a la tasa de mortalidad femenina. El indicador de la tasa de mortalidad se mide sobre 1000 adultos del sexo correspondiente. Es decir, la tasa de mortalidad de mujeres adultas es la tasa de mortalidad de mujeres adultas cada 1000 mujeres adultas; lo mismo se aplica para el hombre.

Además, como ya se mostró en graficos anteriores, en todos los países existe una tendencia a la baja de dichos indicadores. Bolivia y Perú son los países que con una tasa de mortalidad femenina y masculina menos heterogeneas por sexo. Estos dos países, junto con Chile, muestran una caída sostenida en todo el período de ambas tasas.

In [ ]:
unemployment_features = ["Desempleo, Varones", "Desempleo, Mujeres"]

for Features in unemployment_features:
    plt.figure(figsize=(20,10))
    sns.lineplot(data=preferred_features_dataset_alt[preferred_features_dataset_alt["features"]==Features], x="Year", y="value", hue="Country Name")
    plt.title("Series Temporales del \n" + str(Features) + " por país")
    plt.xticks(rotation=90)
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., fontsize= 'xx-small')

In [ ]:
countries = dataset['Country Name'].drop_duplicates().to_list()
unemployment_dataset = preferred_features_dataset_alt[preferred_features_dataset_alt["features"].isin(unemployment_features)]

for country in countries:
    plt.figure(figsize=(20,10))
    sns.lineplot(data=unemployment_dataset[unemployment_dataset["Country Name"]==country], x="Year", y="value", hue="features")
    plt.title("Series Temporales de Desempleo por sexo en \n" + str(country))
    plt.xticks(rotation=90)
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., fontsize= 'xx-small')

En todos los países el desempleo femenino es mayor al masculino y pareciera respoden a los mismos shocks dado que tiene un comportamiento similar. Para algunos países en particular, la diferencia positiva entre desempleo femenino y masculino ha disminuido en la última decada, como es el caso de Bolivia, Paraguay y Chile.


In [ ]:
poverty_features = ["Tasa de incidencia de la pobreza"]

for Features in poverty_features:
    plt.figure(figsize=(20,10))
    sns.lineplot(data=preferred_features_dataset_alt[preferred_features_dataset_alt["features"]==Features], x="Year", y="value", hue="Country Name")
    plt.title("Serie Temporal de la \n" + str(Features) + " por país.")
    plt.xticks(rotation=90)
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., fontsize= 'xx-small')

Respecto a la tasa de incidencia de la pobreza, a partir de los años en que empieza a existir información para analizar, se observa en todos los países una tendencia a la baja.

In [ ]:
countries = dataset['Country Name'].drop_duplicates().to_list()
poverty_dataset = preferred_features_dataset_alt[preferred_features_dataset_alt["features"].isin(poverty_features)]

for country in countries:
      if poverty_dataset[poverty_dataset["Country Name"]== country]["value"].count()!=0:
        plt.figure(figsize=(20,10))
        sns.lineplot(data=poverty_dataset[poverty_dataset["Country Name"]==country], x="Year", y="value", hue="features")
        plt.title("Serie Temporal de la Tasa de Incidencia de la Pobreza de \n" + str(country))
        plt.xticks(rotation=90)
        plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
      else:
        print("No existen datos para el país: " + str(country))

Como puede observarse:
* No existe información de la tasa de incidencia de la pobreza en todo el período en análsis para:
    * Brasil
    * Guyana
    * Suriname
* El país con mas información en el período de análsis es Venezuela, se tiene información desde 1997; sin embargo, no presenta información actualizada para los últimos años  .

* Por su parte, llama la atención la situación de Argentina, que solo tiene información para el período 2016-2018.

### Features compartidos entre indicadores

In [ ]:
#renombrar la columna Country Name
new_df = dataset.rename(columns={'Country Name':'Country'})

#selecciono por un país
select_arg = new_df.loc[new_df['Country'] == 'Argentina']
select_bra = new_df.loc[new_df['Country'] == 'Brasil']
select_uru = new_df.loc[new_df['Country'] == 'Uruguay']
select_par = new_df.loc[new_df['Country'] == 'Paraguay']
select_ven = new_df.loc[new_df['Country'] == 'Venezuela']
select_bol = new_df.loc[new_df['Country'] == 'Bolivia']
select_chi = new_df.loc[new_df['Country'] == 'Chile']
select_guy = new_df.loc[new_df['Country'] == 'Guyana']
select_sur = new_df.loc[new_df['Country'] == 'Suriname']
select_col = new_df.loc[new_df['Country'] == 'Colombia']
select_ecu = new_df.loc[new_df['Country'] == 'Ecuador']
select_per = new_df.loc[new_df['Country'] == 'Perú']

#busco los features duplicados
duparg = select_arg[select_arg.duplicated(['features'])]
dupbra = select_bra[select_bra.duplicated(['features'])]
dupuru = select_uru[select_uru.duplicated(['features'])]
duppar = select_par[select_par.duplicated(['features'])]
dupven = select_ven[select_ven.duplicated(['features'])]
dupbol = select_bol[select_bol.duplicated(['features'])]
dupchi = select_chi[select_chi.duplicated(['features'])]
dupguy = select_guy[select_guy.duplicated(['features'])]
dupsur = select_sur[select_sur.duplicated(['features'])]
dupcol = select_col[select_col.duplicated(['features'])]
dupecu = select_ecu[select_ecu.duplicated(['features'])]
dupper = select_per[select_per.duplicated(['features'])]

In [ ]:
print(duparg['features'])

Existen 89 features que se repiten en al menos dos indicadores para cada uno de los paises.

En general las features que se repiten son variables que aportan al análisis de los indicadores de Género, Educación y Protección Social.

## CONCLUSIONES

A lo largo de la notebook se ha explorado el comportamiento de los indicadores sociales de género, salud, educación, protección social, pobreza y ambiente en los distintos países del MERCOSUR, poniendo especial énfasis en aquellos features por indicador con mayor proporción de datos.

Dado que el objetivo último del trabajo es la proyección de los features para los diversos indicadores, se realizó un análisis profundo de los valores faltantes, debido a que la presencia de los mismos interfiere en la predicción y, en el limite, la imposibilita para casos específicos. 

La ausencia de datos es un problema del dataset en general para todos los indicadores, que se acentúa en determinados intervalos de tiempo o, eventualmente, en ciertos países según indicador. A continuación se resume el perfil de cada indicador al respecto, teniendo en cuenta los heatmap graficados. 
-	El indicador en situación más complicada en cuanto a valores faltantes es el de pobreza. La mayor cantidad de NaNs se encuentra en el periodo de tiempo de 1970 a 1992, a la vez que para el año 2019 ningún feature tiene dato. Adicionalmente, como se observa en el heatmap del indicador que distingue entre países, Guyana y Suriname presentan gran cantidad de valores faltantes en todo el intervalo de tiempo considerado, tratandose de países que debieran descartarse para la proyección.
-	Educación es otro indicador con gran cantidad de NaNs. Los mismos se concentran en el intervalo de tiempo de 1970 a 1997, y en el de 2018 a 2019. Además, hay países particulares que son críticos en cuanto a la falta de información para todos los años, como es el caso de Brasil.  
-	El indicador de ambiente evidencia una situación más comprometedora para la predicción, dado que, no sólo presenta gran proporción de NaNs de 1970 a 1990, sino que también presenta gran cantidad de nulos en la última década. Desde el año 2009 hasta la actualidad empieza a crecer la cantidad de datos nulos de los features. Para el año 2013 ya hay una gran cantidad de datos nulos que sigue creciendo hasta el punto en el cual no hay feature que tenga información en el año 2019, lo que condicionará nuestra predicción. En este caso, paralelamente, no hay un país que no presente datos para todo el intervalo de tiempo como sucede en otros indicadores, pero Suriname se encuentra en desventaja en relación al resto de los países. 
-	Para el caso de los indicadores de género y protección social, la mayoría de los valores faltantes se sitúan en las décadas de 1970 y 1980, aumentando la proporción de datos desde 1990.  Ambos indicadores presentan alta proporción de valores nulos para 2019, pero no es una situación tan crítica como para otros indicadores. En relación a la presencia de algún país con mayor cantidad de valores faltantes, para el caso de género no hay un país demasiado delicado, aunque Brasil se encuentra en desventaja respecto al resto de países (en línea con lo que sucede para educación), mientras que para protección social, Suriname y Guyana se encuentran muy comprometidos (en línea con lo que sucede para pobreza).
-	Para el indicador salud, por su parte, se tiene muy poca proporción de valores faltantes salvo para el año 2019 en el que ningún feature tiene dato. Cabe destacar que unos pocos features, los últimos del heatmap general del indicador, presentan el fenómeno de los indicadores de genero y protección social de presentar valores faltantes de 1970 a 1990.

Finalmente, cabe destacar que los features escogidos para la predicción para todos los indicadores, dada la falta de datos para 2019, se proyectarían para tal año, a excepción de los correspondientes al indicador Ambiente para el cual cabria una predicción del año 2018 (

En lo que concierne a la distribución y las medidas de posición de los features elegidos, manifestadas en las funciones de densidad y los boxplot respectivamente, se evidencia gran heterogeneidad entre los países analizados para la mayoría de los indicadores. 

Para algunos indicadores es posible advertir ciertas particularidades entre o para determinados países en cuanto a sus funciones de densidad. En el caso de salud, por ejemplo, se observa cómo, para muchos de los features, Bolivia y Argentina presentan una elevada curtosis en relación al resto de los países. Por otra parte, en el caso de ambiente no se observan tantas diferencias entre países (hay menos heterogeneidad en las funciones de densidad), aunque el país de la región que pareciera tener un comportamiento sustancialmente distinto al de sus pares para muchos features es Suriname.

En relación a los boxplot, es interesante advertir cómo para algunos indicadores son siempre los mismos países los que presentan menor rango de variación de datos (cajas más pequeñas) o mayor rango de variación de datos (cajas más grandes). En género se manifiesta que los países con mayor rango de variación de datos son Bolivia, Brasil, Chile y Perú, mientras que no es posible categorizar un conjunto de países con menor rango de variación. En el caso de salud, a la inversa, se aprecia cómo en términos generales los países con menor rango de variación de datos son Argentina y Bolivia, mientras que el resto de los países presentan cajas disimiles para los diferentes features y no es posible categorizar a aquellos con mayor rango de variación. Ambiente, por su parte, presenta a Brasil, Bolivia y Venezuela como países con mayor dispersión de datos, y a Uruguay y Argentina como países con poca variabilidad. 
